# 1.- Importación de librerías:

In [ ]:
!pip install selenium
!pip install fake_useragent
!pip install requests-html
!pip install webdriver-manager
!pip install --upgrade webdriver-manager

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.2/460.2 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.4/83.4 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 7.7 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.7
    Uninstalling urllib3-2.0.7:
      Successfully uninstalled urllib3-2.0.7


In [ ]:
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from multiprocessing import Pool
import random
import json
import os
import pandas as pd
from datetime import datetime

# 2.-Configuración de opciones de Chrome y Creación del objeto WebDriver:

In [ ]:
# Crear un objeto UserAgent
ua = UserAgent()

# Crear un objeto Options
chrome_options = Options()
chrome_options.add_argument('--headless')  # Ejecución sin cabeza
chrome_options.add_argument("--incognito")
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-setuid-sandbox")
chrome_options.add_argument("--window-size=1920,1080")
chrome_options.add_argument(f'user-agent={ua.random}')
chrome_options.headless = True

#
# Crear la instancia del navegador con las opciones definidas
driver = webdriver.Chrome(options=chrome_options)

# Establecer timeout
driver.set_page_load_timeout(20)

# O establecer timeout implicito para encontrar elementos
driver.implicitly_wait(20)

# 3.-Obtención del HTML de la página con espera explícita:

In [ ]:

# Definir la URL: Se define la URL de la página de Airbnb para Madrid que se desea analizar.
airbnb_url_madrid = f'https://www.airbnb.es/s/Madrid--España/homes?'

# Conexión: Se utiliza Selenium para abrir la URL en un navegador Chrome controlado por el script.
driver.get(airbnb_url_madrid)
time.sleep(10)  # Esperar a que se carguen los datos

# Obtener el HTML de la página
html = driver.page_source

# Parsear el HTML: Se utiliza BeautifulSoup para parsear el contenido de la respuesta HTTP (que es el HTML de la página web) y crear un objeto BeautifulSoup para facilitar la extracción de datos.
soup = BeautifulSoup(html, 'lxml')


In [ ]:
# Imprimir información sobre la respuesta HTTP
print(driver.current_url)  # URL final después de las redirecciones
print(driver.title)        # Título de la página


https://www.airbnb.es/s/Madrid--Espa%C3%B1a/homes
Airbnb | Alquileres vacacionales y alojamientos en Madrid - Comunidad de Madrid


#4.-Extracción de los elementos 15 paginas -botones

## Acceso a Google Drive

-  Guardar automaticamente los csv en el Google *Drive*

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Crear botón para cada una de las 15 paginas

1.- Botón 1 es la primera página no hace falta clic

2.- Botón 2 y 3 se hace con driver.find_element_by_xpath ya que es la segunda pagina y está visible

3.- Botón 4 a la 15 se hace con navegación con WebDriverWait.EC.element_to_be_clickable y contains @arial label Siguiente.

4.- En la mayoría se carga de nuevo la configuración de opciones de Chrome y se crea el objeto WebDriver, para que el botón pueda ser clickeado

In [ ]:
# @title boton 1

# import time # Librería para controlar el tiempo
# from selenium import webdriver # Librería para controlar el navegador
# from selenium.webdriver.chrome.options import Options # Librería para controlar el navegador
# from selenium.webdriver.common.by import By # Librería para controlar el navegador
# from selenium.webdriver.support.ui import WebDriverWait # Librería para controlar el navegador
# from selenium.webdriver.support import expected_conditions as EC #  Librería para controlar el navegador
# from fake_useragent import UserAgent # Librería para controlar el navegador
# import pandas as pd # Librería para controlar el navegador

# # Configuración inicial del WebDriver y opciones
# ua = UserAgent() # Crear un objeto UserAgent
# chrome_options = Options() # Crear un objeto Options
# chrome_options.add_argument('--headless') # Modo headless
# chrome_options.add_argument('--no-sandbox')
# chrome_options.add_argument('--disable-dev-shm-usage')
# chrome_options.add_argument("--disable-extensions")
# chrome_options.add_argument(f'user-agent={ua.random}')
# chrome_options.headless = True #
# driver = webdriver.Chrome(options=chrome_options) # Crear un objeto WebDriver

# URL de la primera página
airbnb_url_madrid = 'https://www.airbnb.es/s/Madrid--España/homes'

# Crear una nueva sesión de navegador

driver.get(airbnb_url_madrid)
time.sleep(10)  # Ajusta este tiempo según sea necesario

# Lista para almacenar la información de cada anuncio en la página actual
anuncios_pisos_pag1 = []
# Esperar a que cargue la página (o la página siguiente después de hacer clic en el botón)
time.sleep(10)


try:
    # Esperar hasta que los anuncios estén presentes en el DOM
    elementos_anunciospag1 = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, 'c4mnd7m'))
    )

    # Esperar hasta que los elementos adicionales estén presentes en el DOM
    precios_noche = WebDriverWait(driver, 30).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, '_1jo4hgw'))
    )
    subtitulos_anuncio = WebDriverWait(driver, 30).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, 't6mzqp7'))
    )
    valoraciones_anuncio = WebDriverWait(driver, 30).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".r1dxllyb.atm_7l_18pqv07.atm_cp_1ts48j8.dir.dir-ltr"))
    )
    titulos_anuncio = WebDriverWait(driver, 30).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, 't1jojoys'))
    )
    precio_total = WebDriverWait(driver, 30).until(
    EC.presence_of_all_elements_located((By.CLASS_NAME, '_tt122m')))

    for index, anunciopag1 in enumerate(elementos_anunciospag1):  ## Recorrer todos los elementos de anuncios en la página deseada.  Index es el índice de cada elemento . Enumerate es el índice de cada elemento
        try:
            # Extracción del título del anuncio actual o 'N/A' si no hay título disponible
            titulo_anuncio = titulos_anuncio[index].text if index < len(titulos_anuncio) else 'N/A'
            subtitulo_anuncio = subtitulos_anuncio[index].text if index < len(subtitulos_anuncio) else 'N/A'
            valoración_anuncio = valoraciones_anuncio[index].text if index < len(valoraciones_anuncio) else 'N/A'
            precio_noche = precios_noche[index].text if index < len(precios_noche) else 'N/A'
            pprecio_total_texto = precio_total[index].text if index < len(precio_total) else 'N/A'

            enlace_anuncio = anunciopag1.find_element(By.TAG_NAME, 'a')
            href_anuncio = enlace_anuncio.get_attribute('href')
            target_anuncio = enlace_anuncio.get_attribute('target')

            print("Texto del anuncio:", anunciopag1.text)
            print("Título:", titulo_anuncio)
            print("Subtítulo:", subtitulo_anuncio)
            print("Valoración:", valoración_anuncio)
            print("Precio Noche:", precio_noche)
            print("Precio Total:", precio_total)
            print("Href del anuncio:", href_anuncio)
            print("Target del anuncio:", target_anuncio)
            print("-" * 50)

            anuncios_pisos_pag1.append({  #append es para agregar un elemento a la lista
                'texto': anunciopag1.text,
                'titulo': titulo_anuncio,
                'subtitulo': subtitulo_anuncio,
                'valoración': valoración_anuncio,
                'precio_noche': precio_noche,
                'precio_total':precio_total,
                'href': href_anuncio,
                'target': target_anuncio
            })

        except Exception as e_inner:
            print(f"Error al procesar un anuncio: {e_inner}")

except Exception as e:
    print(f"No se pudo encontrar los elementos de los anuncios: {e}")

# Intentar realizar scraping solo si se obtuvieron enlaces
if anuncios_pisos_pag1:
    try:
        # Realizar scraping para buscar las características específicas
        for anuncio in anuncios_pisos_pag1:    # Verificación de que se hayan extraido anuncios
            driver.get(anuncio['href'])  #Ir a la página del anuncio extraido

            try:
                # Esperar hasta que las características estén presentes en el DOM
                WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "lgx66tx.atm_gi_idpfg4.atm_l8_idpfg4.dir.dir-ltr")))

                # Obtener las características
                caracteristicas = driver.find_elements(By.CLASS_NAME, "lgx66tx.atm_gi_idpfg4.atm_l8_idpfg4.dir.dir-ltr")
                caracteristicas_texto = [c.text for c in caracteristicas]

                # Imprimir las características encontradas (opcional)
                print(f"Características para {anuncio['href']}: {caracteristicas_texto}")

                # Añadir las características al diccionario
                anuncio['caracteristicas'] = caracteristicas_texto

            except Exception as e:
                print(f"No se pudo realizar scraping de características para {anuncio['href']}: {e}")

    except Exception as e:
        print(f"No se pudo realizar scraping de características: {e}")
else:
    print("No se obtuvieron enlaces, no se realiza scraping.")


Texto del anuncio: Apartamento en Tetuán
Plaza Castilla Apartments
Anfitrión profesional
18–23 feb
115 € 
71 € 
noche
71 € por noche, inicialmente 115 €
·
354 € en total
354 € en total
Consulta el desglose del precio
4,39 (33)
Título: Apartamento en Tetuán
Subtítulo: Plaza Castilla Apartments
Valoración: 4,39 (33)
Precio Noche: 115 € 
71 € 
noche
Precio Total: [<selenium.webdriver.remote.webelement.WebElement (session="d251d82c13a355dfd55d5df7a3b067bb", element="A7C60D314C090E1333307E41A1FE86E8_element_438")>, <selenium.webdriver.remote.webelement.WebElement (session="d251d82c13a355dfd55d5df7a3b067bb", element="A7C60D314C090E1333307E41A1FE86E8_element_439")>, <selenium.webdriver.remote.webelement.WebElement (session="d251d82c13a355dfd55d5df7a3b067bb", element="A7C60D314C090E1333307E41A1FE86E8_element_440")>, <selenium.webdriver.remote.webelement.WebElement (session="d251d82c13a355dfd55d5df7a3b067bb", element="A7C60D314C090E1333307E41A1FE86E8_element_441")>, <selenium.webdriver.remote.w

In [ ]:
#@title boton 1 guardar
from datetime import datetime
        # Guardar los resultados en un DataFrame
df = pd.DataFrame(anuncios_pisos_pag1)

# Limpiar los saltos de línea y comas en las columnas de texto
df['titulo'] = df['titulo'].str.replace('\n', ' ').str.replace(',', '')
df['subtitulo'] = df['subtitulo'].str.replace('\n', ' ').str.replace(',', '')
# Repetir para cualquier otra columna que pueda tener este problema


# Ruta donde deseas guardar el archivo CSV en Google Drive (nombre único con marca de tiempo)
timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
nombre_archivo_unico = f'anuncios_airbnb_madrid_pag1_{timestamp}.csv'
ruta_guardado_unico = f'/content/drive/MyDrive/Hosting/Hosting/Analisis Scrapping/csv/{nombre_archivo_unico}'

# Guardar el DataFrame en un archivo CSV único basado en la marca de tiempo en Google Drive
df.to_csv(ruta_guardado_unico, sep='\t', encoding='utf-16', index=False)
print(f"Archivo CSV único con marca de tiempo guardado en: {ruta_guardado_unico}")


In [ ]:
#@title boton 2
# import time # Librería para controlar el tiempo
# from selenium import webdriver # Librería para controlar el navegador
# from selenium.webdriver.chrome.options import Options # Librería para controlar el navegador
# from selenium.webdriver.common.by import By # Librería para controlar el navegador
# from selenium.webdriver.support.ui import WebDriverWait # Librería para controlar el navegador
# from selenium.webdriver.support import expected_conditions as EC #  Librería para controlar el navegador
# from fake_useragent import UserAgent # Librería para controlar el navegador
# import pandas as pd # Librería para controlar el navegador

# # Configuración inicial del WebDriver y opciones
# ua = UserAgent() # Crear un objeto UserAgent
# chrome_options = Options() # Crear un objeto Options
# chrome_options.add_argument('--headless') # Modo headless
# chrome_options.add_argument('--no-sandbox')
# chrome_options.add_argument('--disable-dev-shm-usage')
# chrome_options.add_argument("--disable-extensions")
# chrome_options.add_argument(f'user-agent={ua.random}')
# chrome_options.headless = True #
# driver = webdriver.Chrome(options=chrome_options) # Crear un objeto WebDriver


# URL de la primera página
airbnb_url_madrid = 'https://www.airbnb.es/s/Madrid--España/homes'

# Cargar la primera página
driver.get(airbnb_url_madrid)

# Esperar a que cargue la página
time.sleep(5)

# Scroll para asegurarse de que el enlace de la página siguiente sea visible
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")


# Encontrar y hacer clic en el enlace para ir a la siguiente página
try:
    boton_2 = driver.find_element(By.XPATH, "//a[contains(@class, 'atm_1y33qqm_1ggndnn') and contains(text(), '2')]")

    time.sleep(5)
    boton_2.click()
except Exception as e:
    print(f"No se pudo hacer clic en el botón 2: {e}")

# Esperar a que cargue la página siguiente
time.sleep(5)

# Imprimir la URL actual después de hacer clic en el botón de siguiente
print("URL actual después de hacer clic en el botón 2:", driver.current_url)

# Esperar a que cargue la página 4 (o la página siguiente después de hacer clic en el botón)
time.sleep(10)
# Lista para almacenar la información de cada anuncio en la página actual
anuncios_pisos_pag2 = []

try:
    # Esperar hasta que los anuncios estén presentes en el DOM
    elementos_anunciospag2 = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, 'c4mnd7m'))
    )

    # Esperar hasta que los elementos adicionales estén presentes en el DOM
    precios_noche = WebDriverWait(driver, 30).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, '_1jo4hgw'))
    )
    subtitulos_anuncio = WebDriverWait(driver, 30).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, 't6mzqp7'))
    )
    valoraciones_anuncio = WebDriverWait(driver, 30).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".r1dxllyb.atm_7l_18pqv07.atm_cp_1ts48j8.dir.dir-ltr"))
    )
    titulos_anuncio = WebDriverWait(driver, 30).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, 't1jojoys'))
    )
    precio_total = WebDriverWait(driver, 30).until(
    EC.presence_of_all_elements_located((By.CLASS_NAME, '_tt122m')))

    for index, anunciopag2 in enumerate(elementos_anunciospag2):
        try:
            # Extracción del título del anuncio actual o 'N/A' si no hay título disponible
            titulo_anuncio = titulos_anuncio[index].text if index < len(titulos_anuncio) else 'N/A'
            subtitulo_anuncio = subtitulos_anuncio[index].text if index < len(subtitulos_anuncio) else 'N/A'
            valoración_anuncio = valoraciones_anuncio[index].text if index < len(valoraciones_anuncio) else 'N/A'
            precio_noche = precios_noche[index].text if index < len(precios_noche) else 'N/A'
            pprecio_total_texto = precio_total[index].text if index < len(precio_total) else 'N/A'

            enlace_anuncio = anunciopag2.find_element(By.TAG_NAME, 'a')
            href_anuncio = enlace_anuncio.get_attribute('href')
            target_anuncio = enlace_anuncio.get_attribute('target')

            print("Texto del anuncio:", anunciopag2.text)
            print("Título:", titulo_anuncio)
            print("Subtítulo:", subtitulo_anuncio)
            print("Valoración:", valoración_anuncio)
            print("Precio Noche:", precio_noche)
            print("Precio Total:", precio_total)
            print("Href del anuncio:", href_anuncio)
            print("Target del anuncio:", target_anuncio)
            print("-" * 50)

            anuncios_pisos_pag2.append({   #append es para agregar un elemento a la lista
                'texto': anunciopag2.text,
                'titulo': titulo_anuncio,
                'subtitulo': subtitulo_anuncio,
                'valoración': valoración_anuncio,
                'precio_noche': precio_noche,
                'precio_total':precio_total,
                'href': href_anuncio,
                'target': target_anuncio
            })

        except Exception as e_inner:
            print(f"Error al procesar un anuncio: {e_inner}")

except Exception as e:
    print(f"No se pudo encontrar los elementos de los anuncios: {e}")
    print("URL actual cuando ocurrió el error:", driver.current_url)

# Intentar realizar scraping solo si se obtuvieron enlaces
if anuncios_pisos_pag2:    # Verificación de que se hayan extraido anuncios de la página
    try:
        # Realizar scraping para buscar las características específicas
        for anuncio in anuncios_pisos_pag2:
            driver.get(anuncio['href'])

            try:
                # Esperar hasta que las características estén presentes en el DOM
                WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "lgx66tx.atm_gi_idpfg4.atm_l8_idpfg4.dir.dir-ltr")))

                # Obtener las características
                caracteristicas = driver.find_elements(By.CLASS_NAME, "lgx66tx.atm_gi_idpfg4.atm_l8_idpfg4.dir.dir-ltr")
                caracteristicas_texto = [c.text for c in caracteristicas]

                # Imprimir las características encontradas (opcional)
                print(f"Características para {anuncio['href']}: {caracteristicas_texto}")

                # Añadir las características al diccionario
                anuncio['caracteristicas'] = caracteristicas_texto

            except Exception as e:
                print(f"No se pudo realizar scraping de características para {anuncio['href']}: {e}")

        # Guardar los resultados en un DataFrame
        df = pd.DataFrame(anuncios_pisos_pag2)

    except Exception as e:
        print(f"No se pudo realizar scraping de características: {e}")
else:
    print("No se obtuvieron enlaces, no se realiza scraping.")


URL actual después de hacer clic en el botón 2: https://www.airbnb.es/s/Madrid--Espa%C3%B1a/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&query=Madrid%2C%20Espa%C3%B1a&place_id=ChIJgTwKgJcpQg0RaSKMYcHeNsQ&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2024-03-01&monthly_length=3&monthly_end_date=2024-06-01&price_filter_input_type=0&price_filter_num_nights=5&channel=EXPLORE&federated_search_session_id=ec40dad7-5b5a-401a-92e3-58c0f1db9930&search_type=unknown&pagination_search=true&cursor=eyJzZWN0aW9uX29mZnNldCI6MywiaXRlbXNfb2Zmc2V0IjoxOCwidmVyc2lvbiI6MX0%3D
Texto del anuncio: Recomendación del viajero
Apartamento en Arganzuela
Madrid Centro Estudio
Anfitrión particular
18–23 feb
117 € 
noche
117 € por noche
·
585 € en total
585 € en total
Consulta el desglose del precio
5,0 (29)
Título: Apartamento en Arganzuela
Subtítulo: Madrid Centro Estudio
Valoración: 5,0 (29)
Precio Noche: 117 € 
noche
Precio Total: [<selenium.webdriver.remote.webelement.WebElement (session="d251d8

In [ ]:
#@title boton 2 guardar
from datetime import datetime

# Limpiar los saltos de línea y comas en las columnas de texto
df['titulo'] = df['titulo'].str.replace('\n', ' ').str.replace(',', '')
df['subtitulo'] = df['subtitulo'].str.replace('\n', ' ').str.replace(',', '')
# Repetir para cualquier otra columna que pueda tener este problema


# Ruta donde deseas guardar el archivo CSV en Google Drive (nombre único con marca de tiempo)
timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
nombre_archivo_unico = f'anuncios_airbnb_madrid_pag2_{timestamp}.csv'
ruta_guardado_unico = f'/content/drive/MyDrive/Hosting/Hosting/Analisis Scrapping/csv/{nombre_archivo_unico}'

# Guardar el DataFrame en un archivo CSV único basado en la marca de tiempo en Google Drive
df.to_csv(ruta_guardado_unico, sep='\t', encoding='utf-16', index=False)
print(f"Archivo CSV único con marca de tiempo guardado en: {ruta_guardado_unico}")



Archivo CSV único con marca de tiempo guardado en: /content/drive/MyDrive/Hosting/Hosting/Analisis Scrapping/csv/anuncios_airbnb_madrid_pag2_20240214104354.csv


In [ ]:
#@title boton 3
# import time # Librería para controlar el tiempo
# from selenium import webdriver # Librería para controlar el navegador
# from selenium.webdriver.chrome.options import Options # Librería para controlar el navegador
# from selenium.webdriver.common.by import By # Librería para controlar el navegador
# from selenium.webdriver.support.ui import WebDriverWait # Librería para controlar el navegador
# from selenium.webdriver.support import expected_conditions as EC #  Librería para controlar el navegador
# from fake_useragent import UserAgent # Librería para controlar el navegador
# import pandas as pd # Librería para controlar el navegador

# # Configuración inicial del WebDriver y opciones
# ua = UserAgent() # Crear un objeto UserAgent
# chrome_options = Options() # Crear un objeto Options
# chrome_options.add_argument('--headless') # Modo headless
# chrome_options.add_argument('--no-sandbox')
# chrome_options.add_argument('--disable-dev-shm-usage')
# chrome_options.add_argument("--disable-extensions")
# chrome_options.add_argument(f'user-agent={ua.random}')
# chrome_options.headless = True #
# driver = webdriver.Chrome(options=chrome_options) # Crear un objeto WebDriver


# URL de la primera página
airbnb_url_madrid = 'https://www.airbnb.es/s/Madrid--España/homes'

# Crear una nueva sesión si no hay ninguna
if 'driver' not in locals() or driver is None:
    driver = webdriver.Chrome()

# Cargar la primera página si aún no se ha cargado
if driver.current_url != airbnb_url_madrid:
    driver.get(airbnb_url_madrid)
    time.sleep(5)  # Ajusta este tiempo según sea necesario


# Establecer timeout
driver.set_page_load_timeout(20)

# O establecer timeout para encontrar elementos
driver.implicitly_wait(20)

# Scroll para asegurarse de que el enlace de la página  sea visible
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

# Encontrar y hacer clic en el enlace de la página (el botón)
try:
    boton_3 = driver.find_element(By.XPATH, "//a[contains(@class, 'atm_1y33qqm_1ggndnn') and contains(text(), '3')]")


    boton_3.click()
except Exception as e:
    print(f"No se pudo hacer clic en el botón 3: {e}")

# Esperar a que cargue la página (o la página siguiente después de hacer clic en el botón)
time.sleep(5)

# Imprimir la URL actual después de hacer clic en el botón
print("URL actual después de hacer clic en el botón 3:", driver.current_url)

# Esperar a que cargue la página (o la página siguiente después de hacer clic en el botón)
time.sleep(5)

# Lista para almacenar la información de cada anuncio en la página actual
anuncios_pisos_pag3 = []

try:
    # Esperar hasta que los anuncios estén presentes en el DOM
    elementos_anunciospag3 = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, 'c4mnd7m'))
    )

    # Esperar hasta que los elementos adicionales estén presentes en el DOM
    precios_noche = WebDriverWait(driver, 30).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, '_1jo4hgw'))
    )
    subtitulos_anuncio = WebDriverWait(driver, 30).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, 't6mzqp7'))
    )
    valoraciones_anuncio = WebDriverWait(driver, 30).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".r1dxllyb.atm_7l_18pqv07.atm_cp_1ts48j8.dir.dir-ltr"))
    )
    titulos_anuncio = WebDriverWait(driver, 30).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, 't1jojoys'))
    )
    precio_total = WebDriverWait(driver, 30).until(
    EC.presence_of_all_elements_located((By.CLASS_NAME, '_tt122m')))

    for index, anunciopag3 in enumerate(elementos_anunciospag3):
        try:
            # Extraer el texto de cada elemento de la lista
            titulo_anuncio = titulos_anuncio[index].text if index < len(titulos_anuncio) else 'N/A'
            subtitulo_anuncio = subtitulos_anuncio[index].text if index < len(subtitulos_anuncio) else 'N/A'
            valoración_anuncio = valoraciones_anuncio[index].text if index < len(valoraciones_anuncio) else 'N/A'
            precio_noche = precios_noche[index].text if index < len(precios_noche) else 'N/A'
            pprecio_total_texto = precio_total[index].text if index < len(precio_total) else 'N/A'

            enlace_anuncio = anunciopag3.find_element(By.TAG_NAME, 'a')
            href_anuncio = enlace_anuncio.get_attribute('href')
            target_anuncio = enlace_anuncio.get_attribute('target')

            print("Texto del anuncio:", anunciopag3.text)
            print("Título:", titulo_anuncio)
            print("Subtítulo:", subtitulo_anuncio)
            print("Valoración:", valoración_anuncio)
            print("Precio Noche:", precio_noche)
            print("Precio Total:", precio_total)
            print("Href del anuncio:", href_anuncio)
            print("Target del anuncio:", target_anuncio)
            print("-" * 50)

            anuncios_pisos_pag3.append({
                'texto': anunciopag3.text,
                'titulo': titulo_anuncio,
                'subtitulo': subtitulo_anuncio,
                'valoración': valoración_anuncio,
                'precio_noche': precio_noche,
                'precio_total':precio_total,
                'href': href_anuncio,
                'target': target_anuncio
            })

        except Exception as e_inner:
            print(f"Error al procesar un anuncio: {e_inner}")


except Exception as e:
    print(f"No se pudo encontrar los elementos de los anuncios: {e}")





# Intentar realizar scraping solo si se obtuvieron enlaces
if anuncios_pisos_pag3:
    try:
        # Realizar scraping para buscar las características específicas
        for anuncio in anuncios_pisos_pag3:
            driver.get(anuncio['href'])

            try:
                # Esperar hasta que las características estén presentes en el DOM
                WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "lgx66tx.atm_gi_idpfg4.atm_l8_idpfg4.dir.dir-ltr")))

                # Obtener las características
                caracteristicas = driver.find_elements(By.CLASS_NAME, "lgx66tx.atm_gi_idpfg4.atm_l8_idpfg4.dir.dir-ltr")
                caracteristicas_texto = [c.text for c in caracteristicas]

                # Imprimir las características encontradas (opcional)
                print(f"Características para {anuncio['href']}: {caracteristicas_texto}")

                # Añadir las características al diccionario
                anuncio['caracteristicas'] = caracteristicas_texto

            except Exception as e:
                print(f"No se pudo realizar scraping de características para {anuncio['href']}: {e}")

        # Guardar los resultados en un DataFrame
        df = pd.DataFrame(anuncios_pisos_pag3)

    except Exception as e:
        print(f"No se pudo realizar scraping de características: {e}")
else:
    print("No se obtuvieron enlaces, no se realiza scraping.")


URL actual después de hacer clic en el botón 3: https://www.airbnb.es/s/Madrid--Espa%C3%B1a/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&query=Madrid%2C%20Espa%C3%B1a&place_id=ChIJgTwKgJcpQg0RaSKMYcHeNsQ&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2024-03-01&monthly_length=3&monthly_end_date=2024-06-01&price_filter_input_type=0&price_filter_num_nights=5&channel=EXPLORE&federated_search_session_id=aa084cab-0a57-4260-876e-90c1a650802f&search_type=unknown&pagination_search=true&cursor=eyJzZWN0aW9uX29mZnNldCI6MywiaXRlbXNfb2Zmc2V0IjozNiwidmVyc2lvbiI6MX0%3D
Texto del anuncio: Superanfitrión
Apartamento en Madrid
Tranquilo loft en Bernabéu
Anfitrión particular
18–23 feb
125 € 
93 € 
noche
93 € por noche, inicialmente 125 €
·
466 € en total
466 € en total
Consulta el desglose del precio
4,79 (19)
Título: Apartamento en Madrid
Subtítulo: Tranquilo loft en Bernabéu
Valoración: 4,79 (19)
Precio Noche: 125 € 
93 € 
noche
Precio Total: [<selenium.webdriver.remote.webelement.Web

In [ ]:
#@title boton 3 guardar
from datetime import datetime

# Limpiar los saltos de línea y comas en las columnas de texto
df['titulo'] = df['titulo'].str.replace('\n', ' ').str.replace(',', '')
df['subtitulo'] = df['subtitulo'].str.replace('\n', ' ').str.replace(',', '')
# Repetir para cualquier otra columna que pueda tener este problema


# Ruta donde deseas guardar el archivo CSV en Google Drive (nombre único con marca de tiempo)
timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
nombre_archivo_unico = f'anuncios_airbnb_madrid_pag3_{timestamp}.csv'
ruta_guardado_unico = f'/content/drive/MyDrive/Hosting/Hosting/Analisis Scrapping/csv/{nombre_archivo_unico}'

# Guardar el DataFrame en un archivo CSV único basado en la marca de tiempo en Google Drive
df.to_csv(ruta_guardado_unico, sep='\t', encoding='utf-16', index=False)
print(f"Archivo CSV único con marca de tiempo guardado en: {ruta_guardado_unico}")


Archivo CSV único con marca de tiempo guardado en: /content/drive/MyDrive/Hosting/Hosting/Analisis Scrapping/csv/anuncios_airbnb_madrid_pag3_20240214104515.csv


In [ ]:
#@title boton 4
# import time # Librería para controlar el tiempo
# from selenium import webdriver # Librería para controlar el navegador
# from selenium.webdriver.chrome.options import Options # Librería para controlar el navegador
# from selenium.webdriver.common.by import By # Librería para controlar el navegador
# from selenium.webdriver.support.ui import WebDriverWait # Librería para controlar el navegador
# from selenium.webdriver.support import expected_conditions as EC #  Librería para controlar el navegador
# from fake_useragent import UserAgent # Librería para controlar el navegador
# import pandas as pd # Librería para controlar el navegador

# # Configuración inicial del WebDriver y opciones
# ua = UserAgent() # Crear un objeto UserAgent
# chrome_options = Options() # Crear un objeto Options
# chrome_options.add_argument('--headless') # Modo headless
# chrome_options.add_argument('--no-sandbox')
# chrome_options.add_argument('--disable-dev-shm-usage')
# chrome_options.add_argument("--disable-extensions")
# chrome_options.add_argument(f'user-agent={ua.random}')
# chrome_options.headless = True #
# driver = webdriver.Chrome(options=chrome_options) # Crear un objeto WebDriver


# URL de la primera página de Airbnb
airbnb_url_madrid = 'https://www.airbnb.es/s/Madrid--España/homes'  # URL de la primera página de Airbnb
driver.get(airbnb_url_madrid) # Abrir la página
time.sleep(5) # Esperar 5 segundos
driver.implicitly_wait(20) # Esperar 20 segundos

print("URL inicial:", driver.current_url)  # Imprimir la URL inicial

# Navegación a la página deseada (Página 4)
pagina_deseada = 4 # Página deseada
pagina_actual = 1 # Página actual

while pagina_actual < pagina_deseada: # Realizar clic en el botón Siguiente hasta llegar a la página deseada
    try: # Realizar clic en el botón Siguiente
        boton_siguiente = WebDriverWait(driver, 10).until( # Esperar hasta que se cargue el botón Siguiente en el DOM (10 segundos)
            EC.element_to_be_clickable((By.XPATH, "//a[contains(@aria-label, 'Siguiente')]")) # Buscar el botón Siguiente en el DOM con el atributo 'aria-label'
        )
        boton_siguiente.click() # Hacer clic en el botón
        time.sleep(5) # Esperar 5 segundos
        print(f"Avanzando a la página {pagina_actual + 1}") # Imprimir la página
        pagina_actual += 1 # Aumentar el contador de paginas
        print("URL actual:", driver.current_url)  # Imprimir la URL después de cada cambio de página
    except Exception as e: # No se pudo hacer clic en el botón Siguiente
        print(f"No se pudo hacer clic en el botón Siguiente: {e}")
        break # Salir del bucle si no se puede hacer clic

# Verificación de llegada a la página deseada y extracción de datos
if pagina_actual == pagina_deseada:
    print(f"Se ha llegado a la página {pagina_deseada}.")
    print("URL actual:", driver.current_url)  # Imprimir la URL en la página deseada

    # Lista para almacenar los anuncios
    anuncios_pisos_pag4 = []

    try: # Realizar clic en el botón Siguiente
        WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'c4mnd7m'))) # Esperar hasta que se carguen todos los elementos de anuncios en el DOM (10 segundos)
        elementos_anunciospag4 = driver.find_elements(By.CLASS_NAME, 'c4mnd7m')  # Encontrar todos los elementos de anuncios en la página deseada

        # Esperas para elementos adicionales en el DOM
        precios_noche = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CLASS_NAME, '_1jo4hgw'))) # Esperar hasta que se carguen todos los elementos de anuncios en el DOM (30 segundos)
        subtitulos_anuncio = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 't6mzqp7')))# Esperar hasta que se carguen todos los elementos de anuncios en el DOM (30 segundos)
        valoraciones_anuncio = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".r1dxllyb.atm_7l_18pqv07.atm_cp_1ts48j8.dir.dir-ltr")))# Esperar hasta que se carguen todos los elementos de anuncios en el DOM (30 segundos)
        titulos_anuncio = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 't1jojoys')))# Esperar hasta que se carguen todos los elementos de anuncios en el DOM (30 segundos)
        precios_total = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CLASS_NAME, '_tt122m')))# Esperar hasta que se carguen todos los elementos de anuncios en el DOM (30 segundos)

        for index, anunciopag4 in enumerate(elementos_anunciospag4): # Recorrer todos los elementos de anuncios en la página deseada.  Index es el índice de cada elemento . Enumerate es el índice de cada elemento
            enlace_anuncio = anunciopag4.find_element(By.TAG_NAME, 'a') # Encontrar el enlace del anuncio
            href_anuncio = enlace_anuncio.get_attribute('href') # Extraer la URL del enlace
            target_anuncio = enlace_anuncio.get_attribute('target') # Extraer el atributo target


            # Extraer información usando el índice correspondiente
            titulo = titulos_anuncio[index].text if index < len(titulos_anuncio) else 'N/A' # Extraer el texto de la etiqueta si existe
            subtitulo = subtitulos_anuncio[index].text if index < len(subtitulos_anuncio) else 'N/A'  # Extraer el texto de la etiqueta  si existe
            valoracion = valoraciones_anuncio[index].text if index < len(valoraciones_anuncio) else 'N/A'  #  Extraer el texto de la etiqueta  si existe
            precio_noche = precios_noche[index].text if index < len(precios_noche) else 'N/A'  #    Extraer el texto de la etiqueta si existe
            precio_total = precios_total[index].text if index < len(precios_total) else 'N/A'  # Extraer el texto de la etiqueta  si existe

            anuncios_pisos_pag4.append({   #append es para agregar un elemento a la lista
                'texto': anunciopag4.text,
                'titulo': titulo,
                'subtitulo': subtitulo,
                'valoracion': valoracion,
                'precio_noche': precio_noche,
                'precio_total': precio_total,
                'href': href_anuncio,
                'target': target_anuncio
            })

        # Intentar realizar scraping solo si se obtuvieron enlaces
        if anuncios_pisos_pag4: # Verificación de que se hayan extraido anuncios
            try:
                for anuncio in anuncios_pisos_pag4:  # Recorrer todos los anuncios
                    driver.get(anuncio['href']) # Ir a la página del anuncio extraido
                    WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "lgx66tx.atm_gi_idpfg4.atm_l8_idpfg4.dir.dir-ltr")))  # Esperar a que se carguen las características
                    caracteristicas = driver.find_elements(By.CLASS_NAME, "lgx66tx.atm_gi_idpfg4.atm_l8_idpfg4.dir.dir-ltr") # Encontrar todas las características
                    caracteristicas_texto = [c.text for c in caracteristicas] # Extraer el texto de las características
                    print(f"Características para {anuncio['href']}: {caracteristicas_texto}") # Imprimir las características
                    anuncio['caracteristicas'] = caracteristicas_texto # Agregar las características al anuncio
            except Exception as e: # En caso de error
                print(f"No se pudo realizar scraping de características para {anuncio['href']}: {e}")

            df = pd.DataFrame(anuncios_pisos_pag4) # Crear un DataFrame con los anuncios

    except Exception as e:# En caso de error
        print(f"Error al extraer anuncios: {e}")
else: # Si no se encontraron enlaces
    print("No se obtuvieron enlaces, no se realiza scraping.")



URL inicial: https://www.airbnb.es/s/Madrid--Espa%C3%B1a/homes
Avanzando a la página 2
URL actual: https://www.airbnb.es/s/Madrid--Espa%C3%B1a/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&query=Madrid%2C%20Espa%C3%B1a&place_id=ChIJgTwKgJcpQg0RaSKMYcHeNsQ&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2024-03-01&monthly_length=3&monthly_end_date=2024-06-01&price_filter_input_type=0&price_filter_num_nights=5&channel=EXPLORE&federated_search_session_id=59b3f5fa-40c7-4882-bba7-af648c468ae8&search_type=unknown&pagination_search=true&cursor=eyJzZWN0aW9uX29mZnNldCI6MywiaXRlbXNfb2Zmc2V0IjoxOCwidmVyc2lvbiI6MX0%3D
Avanzando a la página 3
URL actual: https://www.airbnb.es/s/Madrid--Espa%C3%B1a/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&query=Madrid%2C%20Espa%C3%B1a&place_id=ChIJgTwKgJcpQg0RaSKMYcHeNsQ&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2024-03-01&monthly_length=3&monthly_end_date=2024-06-01&price_filter_input_type=0&price_filter_num_nights=5&chann

In [ ]:
#@title boton 4 guardar
from datetime import datetime

# Limpiar los saltos de línea y comas en las columnas de texto
df['titulo'] = df['titulo'].str.replace('\n', ' ').str.replace(',', '')
df['subtitulo'] = df['subtitulo'].str.replace('\n', ' ').str.replace(',', '')
# Repetir para cualquier otra columna que pueda tener este problema


# Ruta donde deseas guardar el archivo CSV en Google Drive (nombre único con marca de tiempo)
timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
nombre_archivo_unico = f'anuncios_airbnb_madrid_pag4_{timestamp}.csv'
ruta_guardado_unico = f'/content/drive/MyDrive/Hosting/Hosting/Analisis Scrapping/csv/{nombre_archivo_unico}'

# Guardar el DataFrame en un archivo CSV único basado en la marca de tiempo en Google Drive
df.to_csv(ruta_guardado_unico, sep='\t', encoding='utf-16', index=False)
print(f"Archivo CSV único con marca de tiempo guardado en: {ruta_guardado_unico}")

Archivo CSV único con marca de tiempo guardado en: /content/drive/MyDrive/Hosting/Hosting/Analisis Scrapping/csv/anuncios_airbnb_madrid_pag4_20240214104707.csv


In [ ]:

#@title boton 5
# import time # Librería para controlar el tiempo
# from selenium import webdriver # Librería para controlar el navegador
# from selenium.webdriver.chrome.options import Options # Librería para controlar el navegador
# from selenium.webdriver.common.by import By # Librería para controlar el navegador
# from selenium.webdriver.support.ui import WebDriverWait # Librería para controlar el navegador
# from selenium.webdriver.support import expected_conditions as EC #  Librería para controlar el navegador
# from fake_useragent import UserAgent # Librería para controlar el navegador
# import pandas as pd # Librería para controlar el navegador

# # Configuración inicial del WebDriver y opciones
# ua = UserAgent() # Crear un objeto UserAgent
# chrome_options = Options() # Crear un objeto Options
# chrome_options.add_argument('--headless') # Modo headless
# chrome_options.add_argument('--no-sandbox')
# chrome_options.add_argument('--disable-dev-shm-usage')
# chrome_options.add_argument("--disable-extensions")
# chrome_options.add_argument(f'user-agent={ua.random}')
# chrome_options.headless = True #
# driver = webdriver.Chrome(options=chrome_options) # Crear un objeto WebDriver

# URL de la primera página de Airbnb
airbnb_url_madrid = 'https://www.airbnb.es/s/Madrid--España/homes'
driver.get(airbnb_url_madrid)
time.sleep(5)
driver.implicitly_wait(20)

print("URL inicial:", driver.current_url)  # Imprimir la URL inicial

# Navegación a la página deseada (Página 5)
pagina_deseada = 5
pagina_actual = 1

while pagina_actual < pagina_deseada:
    try:
        boton_siguiente = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[contains(@aria-label, 'Siguiente')]"))
        )
        boton_siguiente.click()
        time.sleep(5)
        print(f"Avanzando a la página {pagina_actual + 1}")
        pagina_actual += 1
        print("URL actual:", driver.current_url)  # Imprimir la URL después de cada cambio de página
    except Exception as e:
        print(f"No se pudo hacer clic en el botón Siguiente: {e}")
        break

# Verificación de llegada a la página deseada y extracción de datos
if pagina_actual == pagina_deseada:
    print(f"Se ha llegado a la página {pagina_deseada}.")
    print("URL actual:", driver.current_url)  # Imprimir la URL en la página deseada
    anuncios_pisos_pag5 = []

    try:
        WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'c4mnd7m')))
        elementos_anunciospag5 = driver.find_elements(By.CLASS_NAME, 'c4mnd7m')
            # Esperas para elementos adicionales en el DOM
        precios_noche = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CLASS_NAME, '_1jo4hgw')))
        subtitulos_anuncio = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 't6mzqp7')))
        valoraciones_anuncio = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".r1dxllyb.atm_7l_18pqv07.atm_cp_1ts48j8.dir.dir-ltr")))
        titulos_anuncio = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 't1jojoys')))
        precios_total = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CLASS_NAME, '_tt122m')))

        for index, anunciopag5 in enumerate(elementos_anunciospag5):
            enlace_anuncio = anunciopag5.find_element(By.TAG_NAME, 'a')
            href_anuncio = enlace_anuncio.get_attribute('href')
            target_anuncio = enlace_anuncio.get_attribute('target')


            # Extraer información usando el índice correspondiente
            titulo = titulos_anuncio[index].text if index < len(titulos_anuncio) else 'N/A'
            subtitulo = subtitulos_anuncio[index].text if index < len(subtitulos_anuncio) else 'N/A'
            valoracion = valoraciones_anuncio[index].text if index < len(valoraciones_anuncio) else 'N/A'
            precio_noche = precios_noche[index].text if index < len(precios_noche) else 'N/A'
            precio_total = precios_total[index].text if index < len(precios_total) else 'N/A'

            anuncios_pisos_pag5.append({
                'texto': anunciopag5.text,
                'titulo': titulo,
                'subtitulo': subtitulo,
                'valoracion': valoracion,
                'precio_noche': precio_noche,
                'precio_total': precio_total,
                'href': href_anuncio,
                'target': target_anuncio
            })

        # Intentar realizar scraping solo si se obtuvieron enlaces
        if anuncios_pisos_pag5:
            try:
                for anuncio in anuncios_pisos_pag5:
                    driver.get(anuncio['href'])
                    WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "lgx66tx.atm_gi_idpfg4.atm_l8_idpfg4.dir.dir-ltr")))
                    caracteristicas = driver.find_elements(By.CLASS_NAME, "lgx66tx.atm_gi_idpfg4.atm_l8_idpfg4.dir.dir-ltr")
                    caracteristicas_texto = [c.text for c in caracteristicas]
                    print(f"Características para {anuncio['href']}: {caracteristicas_texto}")
                    anuncio['caracteristicas'] = caracteristicas_texto
            except Exception as e:
                print(f"No se pudo realizar scraping de características para {anuncio['href']}: {e}")

            df = pd.DataFrame(anuncios_pisos_pag5)

    except Exception as e:
        print(f"Error al extraer anuncios: {e}")
else:
    print("No se obtuvieron enlaces, no se realiza scraping.")



URL inicial: https://www.airbnb.es/s/Madrid--Espa%C3%B1a/homes
Avanzando a la página 2
URL actual: https://www.airbnb.es/s/Madrid--Espa%C3%B1a/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&query=Madrid%2C%20Espa%C3%B1a&place_id=ChIJgTwKgJcpQg0RaSKMYcHeNsQ&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2024-03-01&monthly_length=3&monthly_end_date=2024-06-01&price_filter_input_type=0&price_filter_num_nights=5&channel=EXPLORE&federated_search_session_id=5bb446c5-bf5a-48be-9b9f-ed10d4447f83&search_type=unknown&pagination_search=true&cursor=eyJzZWN0aW9uX29mZnNldCI6MywiaXRlbXNfb2Zmc2V0IjoxOCwidmVyc2lvbiI6MX0%3D
Avanzando a la página 3
URL actual: https://www.airbnb.es/s/Madrid--Espa%C3%B1a/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&query=Madrid%2C%20Espa%C3%B1a&place_id=ChIJgTwKgJcpQg0RaSKMYcHeNsQ&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2024-03-01&monthly_length=3&monthly_end_date=2024-06-01&price_filter_input_type=0&price_filter_num_nights=5&chann

In [ ]:
#@title boton 5 guardar
from datetime import datetime

# Limpiar los saltos de línea y comas en las columnas de texto
df['titulo'] = df['titulo'].str.replace('\n', ' ').str.replace(',', '')
df['subtitulo'] = df['subtitulo'].str.replace('\n', ' ').str.replace(',', '')
# Repetir para cualquier otra columna que pueda tener este problema


# Ruta donde deseas guardar el archivo CSV en Google Drive (nombre único con marca de tiempo)
timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
nombre_archivo_unico = f'anuncios_airbnb_madrid_pag5_{timestamp}.csv'
ruta_guardado_unico = f'/content/drive/MyDrive/Hosting/Hosting/Analisis Scrapping/csv/{nombre_archivo_unico}'

# Guardar el DataFrame en un archivo CSV único basado en la marca de tiempo en Google Drive
df.to_csv(ruta_guardado_unico, sep='\t', encoding='utf-16', index=False)
print(f"Archivo CSV único con marca de tiempo guardado en: {ruta_guardado_unico}")

Archivo CSV único con marca de tiempo guardado en: /content/drive/MyDrive/Hosting/Hosting/Analisis Scrapping/csv/anuncios_airbnb_madrid_pag5_20240214104936.csv


In [ ]:
#@title botón 6
# import time # Librería para controlar el tiempo
# from selenium import webdriver # Librería para controlar el navegador
# from selenium.webdriver.chrome.options import Options # Librería para controlar el navegador
# from selenium.webdriver.common.by import By # Librería para controlar el navegador
# from selenium.webdriver.support.ui import WebDriverWait # Librería para controlar el navegador
# from selenium.webdriver.support import expected_conditions as EC #  Librería para controlar el navegador
# from fake_useragent import UserAgent # Librería para controlar el navegador
# import pandas as pd # Librería para controlar el navegador

# # Configuración inicial del WebDriver y opciones
# ua = UserAgent() # Crear un objeto UserAgent
# chrome_options = Options() # Crear un objeto Options
# chrome_options.add_argument('--headless') # Modo headless
# chrome_options.add_argument('--no-sandbox')
# chrome_options.add_argument('--disable-dev-shm-usage')
# chrome_options.add_argument("--disable-extensions")
# chrome_options.add_argument(f'user-agent={ua.random}')
# chrome_options.headless = True #
# driver = webdriver.Chrome(options=chrome_options) # Crear un objeto WebDriver



# URL de la primera página de Airbnb
airbnb_url_madrid = 'https://www.airbnb.es/s/Madrid--España/homes'
driver.get(airbnb_url_madrid)
time.sleep(5)
driver.implicitly_wait(20)

print("URL inicial:", driver.current_url)  # Imprimir la URL inicial

# Navegación a la página deseada
pagina_deseada = 6
pagina_actual = 1

while pagina_actual < pagina_deseada:
    try:
        boton_siguiente = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[contains(@aria-label, 'Siguiente')]"))
        )
        boton_siguiente.click()
        time.sleep(5)
        print(f"Avanzando a la página {pagina_actual + 1}")
        pagina_actual += 1
        print("URL actual:", driver.current_url)  # Imprimir la URL después de cada cambio de página
    except Exception as e:
        print(f"No se pudo hacer clic en el botón Siguiente: {e}")
        break

# Verificación de llegada a la página deseada y extracción de datos
if pagina_actual == pagina_deseada:
    print(f"Se ha llegado a la página {pagina_deseada}.")
    print("URL actual:", driver.current_url)  # Imprimir la URL en la página deseada
    anuncios_pisos_pag6 = []

    try:
        WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'c4mnd7m')))
        elementos_anunciospag6 = driver.find_elements(By.CLASS_NAME, 'c4mnd7m')
            # Esperas para elementos adicionales en el DOM
        precios_noche = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CLASS_NAME, '_1jo4hgw')))
        subtitulos_anuncio = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 't6mzqp7')))
        valoraciones_anuncio = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".r1dxllyb.atm_7l_18pqv07.atm_cp_1ts48j8.dir.dir-ltr")))
        titulos_anuncio = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 't1jojoys')))
        precios_total = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CLASS_NAME, '_tt122m')))

        for index, anunciopag6 in enumerate(elementos_anunciospag6):
            enlace_anuncio = anunciopag6.find_element(By.TAG_NAME, 'a')
            href_anuncio = enlace_anuncio.get_attribute('href')
            target_anuncio = enlace_anuncio.get_attribute('target')


            # Extraer información usando el índice correspondiente
            titulo = titulos_anuncio[index].text if index < len(titulos_anuncio) else 'N/A'
            subtitulo = subtitulos_anuncio[index].text if index < len(subtitulos_anuncio) else 'N/A'
            valoracion = valoraciones_anuncio[index].text if index < len(valoraciones_anuncio) else 'N/A'
            precio_noche = precios_noche[index].text if index < len(precios_noche) else 'N/A'
            precio_total = precios_total[index].text if index < len(precios_total) else 'N/A'

            anuncios_pisos_pag6.append({
                'texto': anunciopag6.text,
                'titulo': titulo,
                'subtitulo': subtitulo,
                'valoracion': valoracion,
                'precio_noche': precio_noche,
                'precio_total': precio_total,
                'href': href_anuncio,
                'target': target_anuncio
            })

        # Intentar realizar scraping solo si se obtuvieron enlaces
        if anuncios_pisos_pag6:
            try:
                for anuncio in anuncios_pisos_pag6:
                    driver.get(anuncio['href'])
                    WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "lgx66tx.atm_gi_idpfg4.atm_l8_idpfg4.dir.dir-ltr")))
                    caracteristicas = driver.find_elements(By.CLASS_NAME, "lgx66tx.atm_gi_idpfg4.atm_l8_idpfg4.dir.dir-ltr")
                    caracteristicas_texto = [c.text for c in caracteristicas]
                    print(f"Características para {anuncio['href']}: {caracteristicas_texto}")
                    anuncio['caracteristicas'] = caracteristicas_texto
            except Exception as e:
                print(f"No se pudo realizar scraping de características para {anuncio['href']}: {e}")

            df = pd.DataFrame(anuncios_pisos_pag6)


    except Exception as e:
        print(f"Error al extraer anuncios: {e}")
else:
    print("No se obtuvieron enlaces, no se realiza scraping.")


URL inicial: https://www.airbnb.es/s/Madrid--Espa%C3%B1a/homes
Avanzando a la página 2
URL actual: https://www.airbnb.es/s/Madrid--Espa%C3%B1a/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&query=Madrid%2C%20Espa%C3%B1a&place_id=ChIJgTwKgJcpQg0RaSKMYcHeNsQ&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2024-03-01&monthly_length=3&monthly_end_date=2024-06-01&price_filter_input_type=0&price_filter_num_nights=5&channel=EXPLORE&federated_search_session_id=f8a30b2e-9531-40ad-bd0a-2ebaa8d14dc6&search_type=unknown&pagination_search=true&cursor=eyJzZWN0aW9uX29mZnNldCI6MywiaXRlbXNfb2Zmc2V0IjoxOCwidmVyc2lvbiI6MX0%3D
Avanzando a la página 3
URL actual: https://www.airbnb.es/s/Madrid--Espa%C3%B1a/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&query=Madrid%2C%20Espa%C3%B1a&place_id=ChIJgTwKgJcpQg0RaSKMYcHeNsQ&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2024-03-01&monthly_length=3&monthly_end_date=2024-06-01&price_filter_input_type=0&price_filter_num_nights=5&chann

In [ ]:
#@title boton 6 guardar
from datetime import datetime

# Limpiar los saltos de línea y comas en las columnas de texto
df['titulo'] = df['titulo'].str.replace('\n', ' ').str.replace(',', '')
df['subtitulo'] = df['subtitulo'].str.replace('\n', ' ').str.replace(',', '')
# Repetir para cualquier otra columna que pueda tener este problema


# Ruta donde deseas guardar el archivo CSV en Google Drive (nombre único con marca de tiempo)
timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
nombre_archivo_unico = f'anuncios_airbnb_madrid_pag6_{timestamp}.csv'
ruta_guardado_unico = f'/content/drive/MyDrive/Hosting/Hosting/Analisis Scrapping/csv/{nombre_archivo_unico}'

# Guardar el DataFrame en un archivo CSV único basado en la marca de tiempo en Google Drive
df.to_csv(ruta_guardado_unico, sep='\t', encoding='utf-16', index=False)
print(f"Archivo CSV único con marca de tiempo guardado en: {ruta_guardado_unico}")

Archivo CSV único con marca de tiempo guardado en: /content/drive/MyDrive/Hosting/Hosting/Analisis Scrapping/csv/anuncios_airbnb_madrid_pag6_20240214105244.csv


In [ ]:
#@title boton 7
# import time # Librería para controlar el tiempo
# from selenium import webdriver # Librería para controlar el navegador
# from selenium.webdriver.chrome.options import Options # Librería para controlar el navegador
# from selenium.webdriver.common.by import By # Librería para controlar el navegador
# from selenium.webdriver.support.ui import WebDriverWait # Librería para controlar el navegador
# from selenium.webdriver.support import expected_conditions as EC #  Librería para controlar el navegador
# from fake_useragent import UserAgent # Librería para controlar el navegador
# import pandas as pd # Librería para controlar el navegador

# # Configuración inicial del WebDriver y opciones
# ua = UserAgent() # Crear un objeto UserAgent
# chrome_options = Options() # Crear un objeto Options
# chrome_options.add_argument('--headless') # Modo headless
# chrome_options.add_argument('--no-sandbox')
# chrome_options.add_argument('--disable-dev-shm-usage')
# chrome_options.add_argument("--disable-extensions")
# chrome_options.add_argument(f'user-agent={ua.random}')
# chrome_options.headless = True #
# driver = webdriver.Chrome(options=chrome_options) # Crear un objeto WebDriver


# URL de la primera página de Airbnb
airbnb_url_madrid = 'https://www.airbnb.es/s/Madrid--España/homes'
driver.get(airbnb_url_madrid)
time.sleep(5)
driver.implicitly_wait(20)

print("URL inicial:", driver.current_url)  # Imprimir la URL inicial

# Navegación a la página deseada
pagina_deseada = 7
pagina_actual = 1

while pagina_actual < pagina_deseada:
    try:
        boton_siguiente = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[contains(@aria-label, 'Siguiente')]"))
        )
        boton_siguiente.click()
        time.sleep(5)
        print(f"Avanzando a la página {pagina_actual + 1}")
        pagina_actual += 1
        print("URL actual:", driver.current_url)  # Imprimir la URL después de cada cambio de página
    except Exception as e:
        print(f"No se pudo hacer clic en el botón Siguiente: {e}")
        break

# Verificación de llegada a la página deseada y extracción de datos
if pagina_actual == pagina_deseada:
    print(f"Se ha llegado a la página {pagina_deseada}.")
    print("URL actual:", driver.current_url)  # Imprimir la URL en la página deseada
    anuncios_pisos_pag7 = []

    try:
        WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'c4mnd7m')))
        elementos_anunciospag7 = driver.find_elements(By.CLASS_NAME, 'c4mnd7m')
            # Esperas para elementos adicionales en el DOM
        precios_noche = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CLASS_NAME, '_1jo4hgw')))
        subtitulos_anuncio = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 't6mzqp7')))
        valoraciones_anuncio = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".r1dxllyb.atm_7l_18pqv07.atm_cp_1ts48j8.dir.dir-ltr")))
        titulos_anuncio = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 't1jojoys')))
        precios_total = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CLASS_NAME, '_tt122m')))

        for index, anunciopag7 in enumerate(elementos_anunciospag7):
            enlace_anuncio = anunciopag7.find_element(By.TAG_NAME, 'a')
            href_anuncio = enlace_anuncio.get_attribute('href')
            target_anuncio = enlace_anuncio.get_attribute('target')


            # Extraer información usando el índice correspondiente
            titulo = titulos_anuncio[index].text if index < len(titulos_anuncio) else 'N/A'
            subtitulo = subtitulos_anuncio[index].text if index < len(subtitulos_anuncio) else 'N/A'
            valoracion = valoraciones_anuncio[index].text if index < len(valoraciones_anuncio) else 'N/A'
            precio_noche = precios_noche[index].text if index < len(precios_noche) else 'N/A'
            precio_total = precios_total[index].text if index < len(precios_total) else 'N/A'

            anuncios_pisos_pag7.append({
                'texto': anunciopag7.text,
                'titulo': titulo,
                'subtitulo': subtitulo,
                'valoracion': valoracion,
                'precio_noche': precio_noche,
                'precio_total': precio_total,
                'href': href_anuncio,
                'target': target_anuncio
            })

        # Intentar realizar scraping solo si se obtuvieron enlaces
        if anuncios_pisos_pag7:
            try:
                for anuncio in anuncios_pisos_pag7:
                    driver.get(anuncio['href'])
                    WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "lgx66tx.atm_gi_idpfg4.atm_l8_idpfg4.dir.dir-ltr")))
                    caracteristicas = driver.find_elements(By.CLASS_NAME, "lgx66tx.atm_gi_idpfg4.atm_l8_idpfg4.dir.dir-ltr")
                    caracteristicas_texto = [c.text for c in caracteristicas]
                    print(f"Características para {anuncio['href']}: {caracteristicas_texto}")
                    anuncio['caracteristicas'] = caracteristicas_texto
            except Exception as e:
                print(f"No se pudo realizar scraping de características para {anuncio['href']}: {e}")

            df = pd.DataFrame(anuncios_pisos_pag7)

    except Exception as e:
        print(f"Error al extraer anuncios: {e}")
else:
    print("No se obtuvieron enlaces, no se realiza scraping.")

URL inicial: https://www.airbnb.es/s/Madrid--Espa%C3%B1a/homes
Avanzando a la página 2
URL actual: https://www.airbnb.es/s/Madrid--Espa%C3%B1a/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&query=Madrid%2C%20Espa%C3%B1a&place_id=ChIJgTwKgJcpQg0RaSKMYcHeNsQ&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2024-03-01&monthly_length=3&monthly_end_date=2024-06-01&price_filter_input_type=0&price_filter_num_nights=5&channel=EXPLORE&federated_search_session_id=9793028e-9e3a-47cd-b305-fb268d472c63&search_type=unknown&pagination_search=true&cursor=eyJzZWN0aW9uX29mZnNldCI6MywiaXRlbXNfb2Zmc2V0IjoxOCwidmVyc2lvbiI6MX0%3D
Avanzando a la página 3
URL actual: https://www.airbnb.es/s/Madrid--Espa%C3%B1a/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&query=Madrid%2C%20Espa%C3%B1a&place_id=ChIJgTwKgJcpQg0RaSKMYcHeNsQ&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2024-03-01&monthly_length=3&monthly_end_date=2024-06-01&price_filter_input_type=0&price_filter_num_nights=5&chann

In [ ]:
#@title boton 7 guardar
from datetime import datetime

# Limpiar los saltos de línea y comas en las columnas de texto
df['titulo'] = df['titulo'].str.replace('\n', ' ').str.replace(',', '')
df['subtitulo'] = df['subtitulo'].str.replace('\n', ' ').str.replace(',', '')
# Repetir para cualquier otra columna que pueda tener este problema


# Ruta donde deseas guardar el archivo CSV en Google Drive (nombre único con marca de tiempo)
timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
nombre_archivo_unico = f'anuncios_airbnb_madrid_pag7_{timestamp}.csv'
ruta_guardado_unico = f'/content/drive/MyDrive/Hosting/Hosting/Analisis Scrapping/csv/{nombre_archivo_unico}'

# Guardar el DataFrame en un archivo CSV único basado en la marca de tiempo en Google Drive
df.to_csv(ruta_guardado_unico, sep='\t', encoding='utf-16', index=False)
print(f"Archivo CSV único con marca de tiempo guardado en: {ruta_guardado_unico}")

Archivo CSV único con marca de tiempo guardado en: /content/drive/MyDrive/Hosting/Hosting/Analisis Scrapping/csv/anuncios_airbnb_madrid_pag7_20240214105559.csv


In [ ]:
#@title boton 8
# import time # Librería para controlar el tiempo
# from selenium import webdriver # Librería para controlar el navegador
# from selenium.webdriver.chrome.options import Options # Librería para controlar el navegador
# from selenium.webdriver.common.by import By # Librería para controlar el navegador
# from selenium.webdriver.support.ui import WebDriverWait # Librería para controlar el navegador
# from selenium.webdriver.support import expected_conditions as EC #  Librería para controlar el navegador
# from fake_useragent import UserAgent # Librería para controlar el navegador
# import pandas as pd # Librería para controlar el navegador

# # Configuración inicial del WebDriver y opciones
# ua = UserAgent() # Crear un objeto UserAgent
# chrome_options = Options() # Crear un objeto Options
# chrome_options.add_argument('--headless') # Modo headless
# chrome_options.add_argument('--no-sandbox')
# chrome_options.add_argument('--disable-dev-shm-usage')
# chrome_options.add_argument("--disable-extensions")
# chrome_options.add_argument(f'user-agent={ua.random}')
# chrome_options.headless = True #
# driver = webdriver.Chrome(options=chrome_options) # Crear un objeto WebDriver


# URL de la primera página de Airbnb
airbnb_url_madrid = 'https://www.airbnb.es/s/Madrid--España/homes'
driver.get(airbnb_url_madrid)
time.sleep(5)
driver.implicitly_wait(20)

print("URL inicial:", driver.current_url)  # Imprimir la URL inicial

# Navegación a la página deseada
pagina_deseada = 8
pagina_actual = 1

while pagina_actual < pagina_deseada:
    try:
        boton_siguiente = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[contains(@aria-label, 'Siguiente')]"))
        )
        boton_siguiente.click()
        time.sleep(5)
        print(f"Avanzando a la página {pagina_actual + 1}")
        pagina_actual += 1
        print("URL actual:", driver.current_url)  # Imprimir la URL después de cada cambio de página
    except Exception as e:
        print(f"No se pudo hacer clic en el botón Siguiente: {e}")
        break

# Verificación de llegada a la página deseada y extracción de datos
if pagina_actual == pagina_deseada:
    print(f"Se ha llegado a la página {pagina_deseada}.")
    print("URL actual:", driver.current_url)  # Imprimir la URL en la página deseada
    anuncios_pisos_pag8 = []

    try:
        WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'c4mnd7m')))
        elementos_anunciospag8 = driver.find_elements(By.CLASS_NAME, 'c4mnd7m')
            # Esperas para elementos adicionales en el DOM
        precios_noche = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CLASS_NAME, '_1jo4hgw')))
        subtitulos_anuncio = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 't6mzqp7')))
        valoraciones_anuncio = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".r1dxllyb.atm_7l_18pqv07.atm_cp_1ts48j8.dir.dir-ltr")))
        titulos_anuncio = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 't1jojoys')))
        precios_total = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CLASS_NAME, '_tt122m')))

        for index, anunciopag8 in enumerate(elementos_anunciospag8):
            enlace_anuncio = anunciopag8.find_element(By.TAG_NAME, 'a')
            href_anuncio = enlace_anuncio.get_attribute('href')
            target_anuncio = enlace_anuncio.get_attribute('target')


            # Extraer información usando el índice correspondiente
            titulo = titulos_anuncio[index].text if index < len(titulos_anuncio) else 'N/A'
            subtitulo = subtitulos_anuncio[index].text if index < len(subtitulos_anuncio) else 'N/A'
            valoracion = valoraciones_anuncio[index].text if index < len(valoraciones_anuncio) else 'N/A'
            precio_noche = precios_noche[index].text if index < len(precios_noche) else 'N/A'
            precio_total = precios_total[index].text if index < len(precios_total) else 'N/A'

            anuncios_pisos_pag8.append({
                'texto': anunciopag8.text,
                'titulo': titulo,
                'subtitulo': subtitulo,
                'valoracion': valoracion,
                'precio_noche': precio_noche,
                'precio_total': precio_total,
                'href': href_anuncio,
                'target': target_anuncio
            })

        # Intentar realizar scraping solo si se obtuvieron enlaces
        if anuncios_pisos_pag8:
            try:
                for anuncio in anuncios_pisos_pag8:
                    driver.get(anuncio['href'])
                    WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "lgx66tx.atm_gi_idpfg4.atm_l8_idpfg4.dir.dir-ltr")))
                    caracteristicas = driver.find_elements(By.CLASS_NAME, "lgx66tx.atm_gi_idpfg4.atm_l8_idpfg4.dir.dir-ltr")
                    caracteristicas_texto = [c.text for c in caracteristicas]
                    print(f"Características para {anuncio['href']}: {caracteristicas_texto}")
                    anuncio['caracteristicas'] = caracteristicas_texto
            except Exception as e:
                print(f"No se pudo realizar scraping de características para {anuncio['href']}: {e}")

            df = pd.DataFrame(anuncios_pisos_pag8)


    except Exception as e:
        print(f"Error al extraer anuncios: {e}")
else:
    print("No se obtuvieron enlaces, no se realiza scraping.")

URL inicial: https://www.airbnb.es/s/Madrid--Espa%C3%B1a/homes
Avanzando a la página 2
URL actual: https://www.airbnb.es/s/Madrid--Espa%C3%B1a/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&query=Madrid%2C%20Espa%C3%B1a&place_id=ChIJgTwKgJcpQg0RaSKMYcHeNsQ&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2024-03-01&monthly_length=3&monthly_end_date=2024-06-01&price_filter_input_type=0&price_filter_num_nights=5&channel=EXPLORE&federated_search_session_id=ba552082-3cfd-4e75-a832-ae21f720184c&search_type=unknown&pagination_search=true&cursor=eyJzZWN0aW9uX29mZnNldCI6MywiaXRlbXNfb2Zmc2V0IjoxOCwidmVyc2lvbiI6MX0%3D
Avanzando a la página 3
URL actual: https://www.airbnb.es/s/Madrid--Espa%C3%B1a/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&query=Madrid%2C%20Espa%C3%B1a&place_id=ChIJgTwKgJcpQg0RaSKMYcHeNsQ&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2024-03-01&monthly_length=3&monthly_end_date=2024-06-01&price_filter_input_type=0&price_filter_num_nights=5&chann

In [ ]:
#@title boton 8 guardar
from datetime import datetime

# Limpiar los saltos de línea y comas en las columnas de texto
df['titulo'] = df['titulo'].str.replace('\n', ' ').str.replace(',', '')
df['subtitulo'] = df['subtitulo'].str.replace('\n', ' ').str.replace(',', '')
# Repetir para cualquier otra columna que pueda tener este problema


# Ruta donde deseas guardar el archivo CSV en Google Drive (nombre único con marca de tiempo)
timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
nombre_archivo_unico = f'anuncios_airbnb_madrid_pag8_{timestamp}.csv'
ruta_guardado_unico = f'/content/drive/MyDrive/Hosting/Hosting/Analisis Scrapping/csv/{nombre_archivo_unico}'

# Guardar el DataFrame en un archivo CSV único basado en la marca de tiempo en Google Drive
df.to_csv(ruta_guardado_unico, sep='\t', encoding='utf-16', index=False)
print(f"Archivo CSV único con marca de tiempo guardado en: {ruta_guardado_unico}")

Archivo CSV único con marca de tiempo guardado en: /content/drive/MyDrive/Hosting/Hosting/Analisis Scrapping/csv/anuncios_airbnb_madrid_pag8_20240214105946.csv


In [28]:
#@title boton9
import time # Librería para controlar el tiempo
from selenium import webdriver # Librería para controlar el navegador
from selenium.webdriver.chrome.options import Options # Librería para controlar el navegador
from selenium.webdriver.common.by import By # Librería para controlar el navegador
from selenium.webdriver.support.ui import WebDriverWait # Librería para controlar el navegador
from selenium.webdriver.support import expected_conditions as EC #  Librería para controlar el navegador
from fake_useragent import UserAgent # Librería para controlar el navegador
import pandas as pd # Librería para controlar el navegador

# Configuración inicial del WebDriver y opciones
ua = UserAgent() # Crear un objeto UserAgent
chrome_options = Options() # Crear un objeto Options
chrome_options.add_argument('--headless') # Modo headless
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument(f'user-agent={ua.random}')
chrome_options.headless = True #
driver = webdriver.Chrome(options=chrome_options) # Crear un objeto WebDriver


# URL de la primera página de Airbnb
airbnb_url_madrid = 'https://www.airbnb.es/s/Madrid--España/homes'
driver.get(airbnb_url_madrid)
time.sleep(5)
driver.implicitly_wait(20)

print("URL inicial:", driver.current_url)  # Imprimir la URL inicial

# Navegación a la página deseada
pagina_deseada = 9
pagina_actual = 1

while pagina_actual < pagina_deseada:
    try:
        boton_siguiente = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[contains(@aria-label, 'Siguiente')]"))
        )
        boton_siguiente.click()
        time.sleep(5)
        print(f"Avanzando a la página {pagina_actual + 1}")
        pagina_actual += 1
        print("URL actual:", driver.current_url)  # Imprimir la URL después de cada cambio de página
    except Exception as e:
        print(f"No se pudo hacer clic en el botón Siguiente: {e}")
        break

# Verificación de llegada a la página deseada y extracción de datos
if pagina_actual == pagina_deseada:
    print(f"Se ha llegado a la página {pagina_deseada}.")
    print("URL actual:", driver.current_url)  # Imprimir la URL en la página deseada
    anuncios_pisos_pag9 = []

    try:
        WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'c4mnd7m')))
        elementos_anunciospag9 = driver.find_elements(By.CLASS_NAME, 'c4mnd7m')
            # Esperas para elementos adicionales en el DOM
        precios_noche = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CLASS_NAME, '_1jo4hgw')))
        subtitulos_anuncio = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 't6mzqp7')))
        valoraciones_anuncio = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".r1dxllyb.atm_7l_18pqv07.atm_cp_1ts48j8.dir.dir-ltr")))
        titulos_anuncio = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 't1jojoys')))
        precios_total = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CLASS_NAME, '_tt122m')))

        for index, anunciopag9 in enumerate(elementos_anunciospag9):
            enlace_anuncio = anunciopag9.find_element(By.TAG_NAME, 'a')
            href_anuncio = enlace_anuncio.get_attribute('href')
            target_anuncio = enlace_anuncio.get_attribute('target')


            # Extraer información usando el índice correspondiente
            titulo = titulos_anuncio[index].text if index < len(titulos_anuncio) else 'N/A'
            subtitulo = subtitulos_anuncio[index].text if index < len(subtitulos_anuncio) else 'N/A'
            valoracion = valoraciones_anuncio[index].text if index < len(valoraciones_anuncio) else 'N/A'
            precio_noche = precios_noche[index].text if index < len(precios_noche) else 'N/A'
            precio_total = precios_total[index].text if index < len(precios_total) else 'N/A'

            anuncios_pisos_pag9.append({
                'texto': anunciopag9.text,
                'titulo': titulo,
                'subtitulo': subtitulo,
                'valoracion': valoracion,
                'precio_noche': precio_noche,
                'precio_total': precio_total,
                'href': href_anuncio,
                'target': target_anuncio
            })


        # Intentar realizar scraping solo si se obtuvieron enlaces
        if anuncios_pisos_pag9:
            try:
                for anuncio in anuncios_pisos_pag9:
                    driver.get(anuncio['href'])
                    WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "lgx66tx.atm_gi_idpfg4.atm_l8_idpfg4.dir.dir-ltr")))
                    caracteristicas = driver.find_elements(By.CLASS_NAME, "lgx66tx.atm_gi_idpfg4.atm_l8_idpfg4.dir.dir-ltr")
                    caracteristicas_texto = [c.text for c in caracteristicas]
                    print(f"Características para {anuncio['href']}: {caracteristicas_texto}")
                    anuncio['caracteristicas'] = caracteristicas_texto
            except Exception as e:
                print(f"No se pudo realizar scraping de características para {anuncio['href']}: {e}")

            df = pd.DataFrame(anuncios_pisos_pag9)
            df.to_csv('resultados_airbnb_pag9.csv', index=False)
            print("Resultados finales:", df)

    except Exception as e:
        print(f"Error al extraer anuncios: {e}")
else:
    print("No se obtuvieron enlaces, no se realiza scraping.")

# driver.quit()


URL inicial: https://www.airbnb.es/s/Madrid--Espa%C3%B1a/homes
Avanzando a la página 2
URL actual: https://www.airbnb.es/s/Madrid--Espa%C3%B1a/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&query=Madrid%2C%20Espa%C3%B1a&place_id=ChIJgTwKgJcpQg0RaSKMYcHeNsQ&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2024-03-01&monthly_length=3&monthly_end_date=2024-06-01&price_filter_input_type=0&price_filter_num_nights=5&channel=EXPLORE&federated_search_session_id=11326e1e-e611-4185-9216-f49cc8d23805&search_type=unknown&pagination_search=true&cursor=eyJzZWN0aW9uX29mZnNldCI6MywiaXRlbXNfb2Zmc2V0IjoxOCwidmVyc2lvbiI6MX0%3D
Avanzando a la página 3
URL actual: https://www.airbnb.es/s/Madrid--Espa%C3%B1a/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&query=Madrid%2C%20Espa%C3%B1a&place_id=ChIJgTwKgJcpQg0RaSKMYcHeNsQ&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2024-03-01&monthly_length=3&monthly_end_date=2024-06-01&price_filter_input_type=0&price_filter_num_nights=5&chann

In [29]:
#@title boton 9 guardar
from datetime import datetime

# Limpiar los saltos de línea y comas en las columnas de texto
df['titulo'] = df['titulo'].str.replace('\n', ' ').str.replace(',', '')
df['subtitulo'] = df['subtitulo'].str.replace('\n', ' ').str.replace(',', '')
# Repetir para cualquier otra columna que pueda tener este problema


# Ruta donde deseas guardar el archivo CSV en Google Drive (nombre único con marca de tiempo)
timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
nombre_archivo_unico = f'anuncios_airbnb_madrid_pag9_{timestamp}.csv'
ruta_guardado_unico = f'/content/drive/MyDrive/Hosting/Hosting/Analisis Scrapping/csv/{nombre_archivo_unico}'

# Guardar el DataFrame en un archivo CSV único basado en la marca de tiempo en Google Drive
df.to_csv(ruta_guardado_unico, sep='\t', encoding='utf-16', index=False)
print(f"Archivo CSV único con marca de tiempo guardado en: {ruta_guardado_unico}")

Archivo CSV único con marca de tiempo guardado en: /content/drive/MyDrive/Hosting/Hosting/Analisis Scrapping/csv/anuncios_airbnb_madrid_pag9_20240214111508.csv


In [30]:
#@title botón 10
# import time # Librería para controlar el tiempo
# from selenium import webdriver # Librería para controlar el navegador
# from selenium.webdriver.chrome.options import Options # Librería para controlar el navegador
# from selenium.webdriver.common.by import By # Librería para controlar el navegador
# from selenium.webdriver.support.ui import WebDriverWait # Librería para controlar el navegador
# from selenium.webdriver.support import expected_conditions as EC #  Librería para controlar el navegador
# from fake_useragent import UserAgent # Librería para controlar el navegador
# import pandas as pd # Librería para controlar el navegador

# # Configuración inicial del WebDriver y opciones
# ua = UserAgent() # Crear un objeto UserAgent
# chrome_options = Options() # Crear un objeto Options
# chrome_options.add_argument('--headless') # Modo headless
# chrome_options.add_argument('--no-sandbox')
# chrome_options.add_argument('--disable-dev-shm-usage')
# chrome_options.add_argument("--disable-extensions")
# chrome_options.add_argument(f'user-agent={ua.random}')
# chrome_options.headless = True #
# driver = webdriver.Chrome(options=chrome_options) # Crear un objeto WebDriver

# URL de la primera página de Airbnb
airbnb_url_madrid = 'https://www.airbnb.es/s/Madrid--España/homes'
driver.get(airbnb_url_madrid)
time.sleep(5)
driver.implicitly_wait(20)

print("URL inicial:", driver.current_url)  # Imprimir la URL inicial

# Navegación a la página deseada
pagina_deseada = 10
pagina_actual = 1

while pagina_actual < pagina_deseada:
    try:
        boton_siguiente = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[contains(@aria-label, 'Siguiente')]"))
        )
        boton_siguiente.click()
        time.sleep(5)
        print(f"Avanzando a la página {pagina_actual + 1}")
        pagina_actual += 1
        print("URL actual:", driver.current_url)  # Imprimir la URL después de cada cambio de página
    except Exception as e:
        print(f"No se pudo hacer clic en el botón Siguiente: {e}")
        break

# Verificación de llegada a la página deseada y extracción de datos
if pagina_actual == pagina_deseada:
    print(f"Se ha llegado a la página {pagina_deseada}.")
    print("URL actual:", driver.current_url)  # Imprimir la URL en la página deseada
    anuncios_pisos_pag10 = []


    try:
        WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'c4mnd7m')))
        elementos_anunciospag10 = driver.find_elements(By.CLASS_NAME, 'c4mnd7m')
            # Esperas para elementos adicionales en el DOM
        precios_noche = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CLASS_NAME, '_1jo4hgw')))
        subtitulos_anuncio = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 't6mzqp7')))
        valoraciones_anuncio = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".r1dxllyb.atm_7l_18pqv07.atm_cp_1ts48j8.dir.dir-ltr")))
        titulos_anuncio = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 't1jojoys')))
        precios_total = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CLASS_NAME, '_tt122m')))

        for index, anunciopag10 in enumerate(elementos_anunciospag10):
            enlace_anuncio = anunciopag10.find_element(By.TAG_NAME, 'a')
            href_anuncio = enlace_anuncio.get_attribute('href')
            target_anuncio = enlace_anuncio.get_attribute('target')


            # Extraer información usando el índice correspondiente
            titulo = titulos_anuncio[index].text if index < len(titulos_anuncio) else 'N/A'
            subtitulo = subtitulos_anuncio[index].text if index < len(subtitulos_anuncio) else 'N/A'
            valoracion = valoraciones_anuncio[index].text if index < len(valoraciones_anuncio) else 'N/A'
            precio_noche = precios_noche[index].text if index < len(precios_noche) else 'N/A'
            precio_total = precios_total[index].text if index < len(precios_total) else 'N/A'

            anuncios_pisos_pag10.append({
                'texto': anunciopag10.text,
                'titulo': titulo,
                'subtitulo': subtitulo,
                'valoracion': valoracion,
                'precio_noche': precio_noche,
                'precio_total': precio_total,
                'href': href_anuncio,
                'target': target_anuncio
            })

        # Intentar realizar scraping solo si se obtuvieron enlaces
        if anuncios_pisos_pag10:
            try:
                for anuncio in anuncios_pisos_pag10:
                    driver.get(anuncio['href'])
                    WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "lgx66tx.atm_gi_idpfg4.atm_l8_idpfg4.dir.dir-ltr")))
                    caracteristicas = driver.find_elements(By.CLASS_NAME, "lgx66tx.atm_gi_idpfg4.atm_l8_idpfg4.dir.dir-ltr")
                    caracteristicas_texto = [c.text for c in caracteristicas]
                    print(f"Características para {anuncio['href']}: {caracteristicas_texto}")
                    anuncio['caracteristicas'] = caracteristicas_texto
            except Exception as e:
                print(f"No se pudo realizar scraping de características para {anuncio['href']}: {e}")

            df = pd.DataFrame(anuncios_pisos_pag10)
            df.to_csv('resultados_airbnb_pag10.csv', index=False)
            print("Resultados finales:", df)

    except Exception as e:
        print(f"Error al extraer anuncios: {e}")
else:
    print("No se obtuvieron enlaces, no se realiza scraping.")



URL inicial: https://www.airbnb.es/s/Madrid--Espa%C3%B1a/homes
Avanzando a la página 2
URL actual: https://www.airbnb.es/s/Madrid--Espa%C3%B1a/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&query=Madrid%2C%20Espa%C3%B1a&place_id=ChIJgTwKgJcpQg0RaSKMYcHeNsQ&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2024-03-01&monthly_length=3&monthly_end_date=2024-06-01&price_filter_input_type=0&price_filter_num_nights=5&channel=EXPLORE&federated_search_session_id=711c52ca-789d-497c-b9d6-00c258ee0e21&search_type=unknown&pagination_search=true&cursor=eyJzZWN0aW9uX29mZnNldCI6MywiaXRlbXNfb2Zmc2V0IjoxOCwidmVyc2lvbiI6MX0%3D
Avanzando a la página 3
URL actual: https://www.airbnb.es/s/Madrid--Espa%C3%B1a/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&query=Madrid%2C%20Espa%C3%B1a&place_id=ChIJgTwKgJcpQg0RaSKMYcHeNsQ&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2024-03-01&monthly_length=3&monthly_end_date=2024-06-01&price_filter_input_type=0&price_filter_num_nights=5&chann

In [31]:
#@title boton 10 guardar
from datetime import datetime

# Limpiar los saltos de línea y comas en las columnas de texto
df['titulo'] = df['titulo'].str.replace('\n', ' ').str.replace(',', '')
df['subtitulo'] = df['subtitulo'].str.replace('\n', ' ').str.replace(',', '')
# Repetir para cualquier otra columna que pueda tener este problema


# Ruta donde deseas guardar el archivo CSV en Google Drive (nombre único con marca de tiempo)
timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
nombre_archivo_unico = f'anuncios_airbnb_madrid_pag10_{timestamp}.csv'
ruta_guardado_unico = f'/content/drive/MyDrive/Hosting/Hosting/Analisis Scrapping/csv/{nombre_archivo_unico}'

# Guardar el DataFrame en un archivo CSV único basado en la marca de tiempo en Google Drive
df.to_csv(ruta_guardado_unico, sep='\t', encoding='utf-16', index=False)
print(f"Archivo CSV único con marca de tiempo guardado en: {ruta_guardado_unico}")

Archivo CSV único con marca de tiempo guardado en: /content/drive/MyDrive/Hosting/Hosting/Analisis Scrapping/csv/anuncios_airbnb_madrid_pag10_20240214111654.csv


In [32]:
#@title botón 11
# import time # Librería para controlar el tiempo
# from selenium import webdriver # Librería para controlar el navegador
# from selenium.webdriver.chrome.options import Options # Librería para controlar el navegador
# from selenium.webdriver.common.by import By # Librería para controlar el navegador
# from selenium.webdriver.support.ui import WebDriverWait # Librería para controlar el navegador
# from selenium.webdriver.support import expected_conditions as EC #  Librería para controlar el navegador
# from fake_useragent import UserAgent # Librería para controlar el navegador
# import pandas as pd # Librería para controlar el navegador

# # Configuración inicial del WebDriver y opciones
# ua = UserAgent() # Crear un objeto UserAgent
# chrome_options = Options() # Crear un objeto Options
# chrome_options.add_argument('--headless') # Modo headless
# chrome_options.add_argument('--no-sandbox')
# chrome_options.add_argument('--disable-dev-shm-usage')
# chrome_options.add_argument("--disable-extensions")
# chrome_options.add_argument(f'user-agent={ua.random}')
# chrome_options.headless = True #
# driver = webdriver.Chrome(options=chrome_options) # Crear un objeto WebDriver


# URL de la primera página de Airbnb
airbnb_url_madrid = 'https://www.airbnb.es/s/Madrid--España/homes'
driver.get(airbnb_url_madrid)
time.sleep(5)
driver.implicitly_wait(20)

print("URL inicial:", driver.current_url)  # Imprimir la URL inicial

# Navegación a la página deseada
pagina_deseada = 11
pagina_actual = 1

while pagina_actual < pagina_deseada:
    try:
        boton_siguiente = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[contains(@aria-label, 'Siguiente')]"))
        )
        boton_siguiente.click()
        time.sleep(5)
        print(f"Avanzando a la página {pagina_actual + 1}")
        pagina_actual += 1
        print("URL actual:", driver.current_url)  # Imprimir la URL después de cada cambio de página
    except Exception as e:
        print(f"No se pudo hacer clic en el botón Siguiente: {e}")
        break

# Verificación de llegada a la página deseada y extracción de datos
if pagina_actual == pagina_deseada:
    print(f"Se ha llegado a la página {pagina_deseada}.")
    print("URL actual:", driver.current_url)  # Imprimir la URL en la página deseada
    anuncios_pisos_pag11 = []

    try:
        WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'c4mnd7m')))
        elementos_anunciospag11 = driver.find_elements(By.CLASS_NAME, 'c4mnd7m')
            # Esperas para elementos adicionales en el DOM
        precios_noche = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CLASS_NAME, '_1jo4hgw')))
        subtitulos_anuncio = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 't6mzqp7')))
        valoraciones_anuncio = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".r1dxllyb.atm_7l_18pqv07.atm_cp_1ts48j8.dir.dir-ltr")))
        titulos_anuncio = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 't1jojoys')))
        precios_total = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CLASS_NAME, '_tt122m')))

        for index, anunciopag11 in enumerate(elementos_anunciospag11):
            enlace_anuncio = anunciopag11.find_element(By.TAG_NAME, 'a')
            href_anuncio = enlace_anuncio.get_attribute('href')
            target_anuncio = enlace_anuncio.get_attribute('target')


            # Extraer información usando el índice correspondiente
            titulo = titulos_anuncio[index].text if index < len(titulos_anuncio) else 'N/A'
            subtitulo = subtitulos_anuncio[index].text if index < len(subtitulos_anuncio) else 'N/A'
            valoracion = valoraciones_anuncio[index].text if index < len(valoraciones_anuncio) else 'N/A'
            precio_noche = precios_noche[index].text if index < len(precios_noche) else 'N/A'
            precio_total = precios_total[index].text if index < len(precios_total) else 'N/A'

            anuncios_pisos_pag11.append({
                'texto': anunciopag11.text,
                'titulo': titulo,
                'subtitulo': subtitulo,
                'valoracion': valoracion,
                'precio_noche': precio_noche,
                'precio_total': precio_total,
                'href': href_anuncio,
                'target': target_anuncio
            })

        # Intentar realizar scraping solo si se obtuvieron enlaces
        if anuncios_pisos_pag11:
            try:
                for anuncio in anuncios_pisos_pag11:
                    driver.get(anuncio['href'])
                    WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "lgx66tx.atm_gi_idpfg4.atm_l8_idpfg4.dir.dir-ltr")))
                    caracteristicas = driver.find_elements(By.CLASS_NAME, "lgx66tx.atm_gi_idpfg4.atm_l8_idpfg4.dir.dir-ltr")
                    caracteristicas_texto = [c.text for c in caracteristicas]
                    print(f"Características para {anuncio['href']}: {caracteristicas_texto}")
                    anuncio['caracteristicas'] = caracteristicas_texto
            except Exception as e:
                print(f"No se pudo realizar scraping de características para {anuncio['href']}: {e}")

            df = pd.DataFrame(anuncios_pisos_pag11)
            df.to_csv('resultados_airbnb_pag11.csv', index=False)
            print("Resultados finales:", df)

    except Exception as e:
        print(f"Error al extraer anuncios: {e}")
else:
    print("No se obtuvieron enlaces, no se realiza scraping.")



URL inicial: https://www.airbnb.es/s/Madrid--Espa%C3%B1a/homes
Avanzando a la página 2
URL actual: https://www.airbnb.es/s/Madrid--Espa%C3%B1a/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&query=Madrid%2C%20Espa%C3%B1a&place_id=ChIJgTwKgJcpQg0RaSKMYcHeNsQ&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2024-03-01&monthly_length=3&monthly_end_date=2024-06-01&price_filter_input_type=0&price_filter_num_nights=5&channel=EXPLORE&federated_search_session_id=53551d7d-9e2c-4779-a2b0-42d105938574&search_type=unknown&pagination_search=true&cursor=eyJzZWN0aW9uX29mZnNldCI6MywiaXRlbXNfb2Zmc2V0IjoxOCwidmVyc2lvbiI6MX0%3D
Avanzando a la página 3
URL actual: https://www.airbnb.es/s/Madrid--Espa%C3%B1a/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&query=Madrid%2C%20Espa%C3%B1a&place_id=ChIJgTwKgJcpQg0RaSKMYcHeNsQ&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2024-03-01&monthly_length=3&monthly_end_date=2024-06-01&price_filter_input_type=0&price_filter_num_nights=5&chann

In [33]:
#@title boton 11 guardar
from datetime import datetime

# Limpiar los saltos de línea y comas en las columnas de texto
df['titulo'] = df['titulo'].str.replace('\n', ' ').str.replace(',', '')
df['subtitulo'] = df['subtitulo'].str.replace('\n', ' ').str.replace(',', '')
# Repetir para cualquier otra columna que pueda tener este problema


# Ruta donde deseas guardar el archivo CSV en Google Drive (nombre único con marca de tiempo)
timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
nombre_archivo_unico = f'anuncios_airbnb_madrid_pag11_{timestamp}.csv'
ruta_guardado_unico = f'/content/drive/MyDrive/Hosting/Hosting/Analisis Scrapping/csv/{nombre_archivo_unico}'

# Guardar el DataFrame en un archivo CSV único basado en la marca de tiempo en Google Drive
df.to_csv(ruta_guardado_unico, sep='\t', encoding='utf-16', index=False)
print(f"Archivo CSV único con marca de tiempo guardado en: {ruta_guardado_unico}")

Archivo CSV único con marca de tiempo guardado en: /content/drive/MyDrive/Hosting/Hosting/Analisis Scrapping/csv/anuncios_airbnb_madrid_pag11_20240214111834.csv


In [34]:
#@title botón 12
# import time # Librería para controlar el tiempo
# from selenium import webdriver # Librería para controlar el navegador
# from selenium.webdriver.chrome.options import Options # Librería para controlar el navegador
# from selenium.webdriver.common.by import By # Librería para controlar el navegador
# from selenium.webdriver.support.ui import WebDriverWait # Librería para controlar el navegador
# from selenium.webdriver.support import expected_conditions as EC #  Librería para controlar el navegador
# from fake_useragent import UserAgent # Librería para controlar el navegador
# import pandas as pd # Librería para controlar el navegador

# # Configuración inicial del WebDriver y opciones
# ua = UserAgent() # Crear un objeto UserAgent
# chrome_options = Options() # Crear un objeto Options
# chrome_options.add_argument('--headless') # Modo headless
# chrome_options.add_argument('--no-sandbox')
# chrome_options.add_argument('--disable-dev-shm-usage')
# chrome_options.add_argument("--disable-extensions")
# chrome_options.add_argument(f'user-agent={ua.random}')
# chrome_options.headless = True #
# driver = webdriver.Chrome(options=chrome_options) # Crear un objeto WebDriver

# URL de la primera página de Airbnb
airbnb_url_madrid = 'https://www.airbnb.es/s/Madrid--España/homes'
driver.get(airbnb_url_madrid)
time.sleep(5)
driver.implicitly_wait(20)

print("URL inicial:", driver.current_url)  # Imprimir la URL inicial

# Navegación a la página deseada
pagina_deseada = 12
pagina_actual = 1

while pagina_actual < pagina_deseada:
    try:
        boton_siguiente = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[contains(@aria-label, 'Siguiente')]"))
        )
        boton_siguiente.click()
        time.sleep(5)
        print(f"Avanzando a la página {pagina_actual + 1}")
        pagina_actual += 1
        print("URL actual:", driver.current_url)  # Imprimir la URL después de cada cambio de página
    except Exception as e:
        print(f"No se pudo hacer clic en el botón Siguiente: {e}")
        break

# Verificación de llegada a la página deseada y extracción de datos
if pagina_actual == pagina_deseada:
    print(f"Se ha llegado a la página {pagina_deseada}.")
    print("URL actual:", driver.current_url)  # Imprimir la URL en la página deseada
    anuncios_pisos_pag12 = []

    try:
        WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'c4mnd7m')))
        elementos_anunciospag12 = driver.find_elements(By.CLASS_NAME, 'c4mnd7m')
            # Esperas para elementos adicionales en el DOM
        precios_noche = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CLASS_NAME, '_1jo4hgw')))
        subtitulos_anuncio = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 't6mzqp7')))
        valoraciones_anuncio = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".r1dxllyb.atm_7l_18pqv07.atm_cp_1ts48j8.dir.dir-ltr")))
        titulos_anuncio = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 't1jojoys')))
        precios_total = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CLASS_NAME, '_tt122m')))

        for index, anunciopag12 in enumerate(elementos_anunciospag12):
            enlace_anuncio = anunciopag12.find_element(By.TAG_NAME, 'a')
            href_anuncio = enlace_anuncio.get_attribute('href')
            target_anuncio = enlace_anuncio.get_attribute('target')


            # Extraer información usando el índice correspondiente
            titulo = titulos_anuncio[index].text if index < len(titulos_anuncio) else 'N/A'
            subtitulo = subtitulos_anuncio[index].text if index < len(subtitulos_anuncio) else 'N/A'
            valoracion = valoraciones_anuncio[index].text if index < len(valoraciones_anuncio) else 'N/A'
            precio_noche = precios_noche[index].text if index < len(precios_noche) else 'N/A'
            precio_total = precios_total[index].text if index < len(precios_total) else 'N/A'

            anuncios_pisos_pag12.append({
                'texto': anunciopag12.text,
                'titulo': titulo,
                'subtitulo': subtitulo,
                'valoracion': valoracion,
                'precio_noche': precio_noche,
                'precio_total': precio_total,
                'href': href_anuncio,
                'target': target_anuncio
            })


        # Intentar realizar scraping solo si se obtuvieron enlaces
        if anuncios_pisos_pag12:
            try:
                for anuncio in anuncios_pisos_pag12:
                    driver.get(anuncio['href'])
                    WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "lgx66tx.atm_gi_idpfg4.atm_l8_idpfg4.dir.dir-ltr")))
                    caracteristicas = driver.find_elements(By.CLASS_NAME, "lgx66tx.atm_gi_idpfg4.atm_l8_idpfg4.dir.dir-ltr")
                    caracteristicas_texto = [c.text for c in caracteristicas]
                    print(f"Características para {anuncio['href']}: {caracteristicas_texto}")
                    anuncio['caracteristicas'] = caracteristicas_texto
            except Exception as e:
                print(f"No se pudo realizar scraping de características para {anuncio['href']}: {e}")

            df = pd.DataFrame(anuncios_pisos_pag12)
            df.to_csv('resultados_airbnb_pag12.csv', index=False)
            print("Resultados finales:", df)

    except Exception as e:
        print(f"Error al extraer anuncios: {e}")
else:
    print("No se obtuvieron enlaces, no se realiza scraping.")




URL inicial: https://www.airbnb.es/s/Madrid--Espa%C3%B1a/homes
Avanzando a la página 2
URL actual: https://www.airbnb.es/s/Madrid--Espa%C3%B1a/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&query=Madrid%2C%20Espa%C3%B1a&place_id=ChIJgTwKgJcpQg0RaSKMYcHeNsQ&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2024-03-01&monthly_length=3&monthly_end_date=2024-06-01&price_filter_input_type=0&price_filter_num_nights=5&channel=EXPLORE&federated_search_session_id=bdab0b65-31c8-43f4-872c-c7f63b166835&search_type=unknown&pagination_search=true&cursor=eyJzZWN0aW9uX29mZnNldCI6MywiaXRlbXNfb2Zmc2V0IjoxOCwidmVyc2lvbiI6MX0%3D
Avanzando a la página 3
URL actual: https://www.airbnb.es/s/Madrid--Espa%C3%B1a/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&query=Madrid%2C%20Espa%C3%B1a&place_id=ChIJgTwKgJcpQg0RaSKMYcHeNsQ&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2024-03-01&monthly_length=3&monthly_end_date=2024-06-01&price_filter_input_type=0&price_filter_num_nights=5&chann

In [35]:
#@title boton 12 guardar
from datetime import datetime

# Limpiar los saltos de línea y comas en las columnas de texto
df['titulo'] = df['titulo'].str.replace('\n', ' ').str.replace(',', '')
df['subtitulo'] = df['subtitulo'].str.replace('\n', ' ').str.replace(',', '')
# Repetir para cualquier otra columna que pueda tener este problema


# Ruta donde deseas guardar el archivo CSV en Google Drive (nombre único con marca de tiempo)
timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
nombre_archivo_unico = f'anuncios_airbnb_madrid_pag12_{timestamp}.csv'
ruta_guardado_unico = f'/content/drive/MyDrive/Hosting/Hosting/Analisis Scrapping/csv/{nombre_archivo_unico}'

# Guardar el DataFrame en un archivo CSV único basado en la marca de tiempo en Google Drive
df.to_csv(ruta_guardado_unico, sep='\t', encoding='utf-16', index=False)
print(f"Archivo CSV único con marca de tiempo guardado en: {ruta_guardado_unico}")

Archivo CSV único con marca de tiempo guardado en: /content/drive/MyDrive/Hosting/Hosting/Analisis Scrapping/csv/anuncios_airbnb_madrid_pag12_20240214112038.csv


In [36]:
#@title botón 13
# import time # Librería para controlar el tiempo
# from selenium import webdriver # Librería para controlar el navegador
# from selenium.webdriver.chrome.options import Options # Librería para controlar el navegador
# from selenium.webdriver.common.by import By # Librería para controlar el navegador
# from selenium.webdriver.support.ui import WebDriverWait # Librería para controlar el navegador
# from selenium.webdriver.support import expected_conditions as EC #  Librería para controlar el navegador
# from fake_useragent import UserAgent # Librería para controlar el navegador
# import pandas as pd # Librería para controlar el navegador

# # Configuración inicial del WebDriver y opciones
# ua = UserAgent() # Crear un objeto UserAgent
# chrome_options = Options() # Crear un objeto Options
# chrome_options.add_argument('--headless') # Modo headless
# chrome_options.add_argument('--no-sandbox')
# chrome_options.add_argument('--disable-dev-shm-usage')
# chrome_options.add_argument("--disable-extensions")
# chrome_options.add_argument(f'user-agent={ua.random}')
# chrome_options.headless = True #
# driver = webdriver.Chrome(options=chrome_options) # Crear un objeto WebDriver


# URL de la primera página de Airbnb
airbnb_url_madrid = 'https://www.airbnb.es/s/Madrid--España/homes'
driver.get(airbnb_url_madrid)
time.sleep(5)
driver.implicitly_wait(20)

print("URL inicial:", driver.current_url)  # Imprimir la URL inicial

# Navegación a la página deseada
pagina_deseada = 13
pagina_actual = 1

while pagina_actual < pagina_deseada:
    try:
        boton_siguiente = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[contains(@aria-label, 'Siguiente')]"))
        )
        boton_siguiente.click()
        time.sleep(5)
        print(f"Avanzando a la página {pagina_actual + 1}")
        pagina_actual += 1
        print("URL actual:", driver.current_url)  # Imprimir la URL después de cada cambio de página
    except Exception as e:
        print(f"No se pudo hacer clic en el botón Siguiente: {e}")
        break

# Verificación de llegada a la página deseada y extracción de datos
if pagina_actual == pagina_deseada:
    print(f"Se ha llegado a la página {pagina_deseada}.")
    print("URL actual:", driver.current_url)  # Imprimir la URL en la página deseada
    anuncios_pisos_pag13 = []

    try:
        WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'c4mnd7m')))
        elementos_anunciospag13 = driver.find_elements(By.CLASS_NAME, 'c4mnd7m')
            # Esperas para elementos adicionales en el DOM
        precios_noche = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CLASS_NAME, '_1jo4hgw')))
        subtitulos_anuncio = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 't6mzqp7')))
        valoraciones_anuncio = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".r1dxllyb.atm_7l_18pqv07.atm_cp_1ts48j8.dir.dir-ltr")))
        titulos_anuncio = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 't1jojoys')))
        precios_total = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CLASS_NAME, '_tt122m')))

        for index, anunciopag13 in enumerate(elementos_anunciospag13):
            enlace_anuncio = anunciopag13.find_element(By.TAG_NAME, 'a')
            href_anuncio = enlace_anuncio.get_attribute('href')
            target_anuncio = enlace_anuncio.get_attribute('target')


            # Extraer información usando el índice correspondiente
            titulo = titulos_anuncio[index].text if index < len(titulos_anuncio) else 'N/A'
            subtitulo = subtitulos_anuncio[index].text if index < len(subtitulos_anuncio) else 'N/A'
            valoracion = valoraciones_anuncio[index].text if index < len(valoraciones_anuncio) else 'N/A'
            precio_noche = precios_noche[index].text if index < len(precios_noche) else 'N/A'
            precio_total = precios_total[index].text if index < len(precios_total) else 'N/A'

            anuncios_pisos_pag13.append({
                'texto': anunciopag13.text,
                'titulo': titulo,
                'subtitulo': subtitulo,
                'valoracion': valoracion,
                'precio_noche': precio_noche,
                'precio_total': precio_total,
                'href': href_anuncio,
                'target': target_anuncio
            })

        # Intentar realizar scraping solo si se obtuvieron enlaces
        if anuncios_pisos_pag13:
            try:
                for anuncio in anuncios_pisos_pag13:
                    driver.get(anuncio['href'])
                    WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "lgx66tx.atm_gi_idpfg4.atm_l8_idpfg4.dir.dir-ltr")))
                    caracteristicas = driver.find_elements(By.CLASS_NAME, "lgx66tx.atm_gi_idpfg4.atm_l8_idpfg4.dir.dir-ltr")
                    caracteristicas_texto = [c.text for c in caracteristicas]
                    print(f"Características para {anuncio['href']}: {caracteristicas_texto}")
                    anuncio['caracteristicas'] = caracteristicas_texto
            except Exception as e:
                print(f"No se pudo realizar scraping de características para {anuncio['href']}: {e}")

            df = pd.DataFrame(anuncios_pisos_pag13)
            df.to_csv('resultados_airbnb_pag13.csv', index=False)
            print("Resultados finales:", df)

    except Exception as e:
        print(f"Error al extraer anuncios: {e}")
else:
    print("No se obtuvieron enlaces, no se realiza scraping.")




URL inicial: https://www.airbnb.es/s/Madrid--Espa%C3%B1a/homes
Avanzando a la página 2
URL actual: https://www.airbnb.es/s/Madrid--Espa%C3%B1a/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&query=Madrid%2C%20Espa%C3%B1a&place_id=ChIJgTwKgJcpQg0RaSKMYcHeNsQ&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2024-03-01&monthly_length=3&monthly_end_date=2024-06-01&price_filter_input_type=0&price_filter_num_nights=5&channel=EXPLORE&federated_search_session_id=88a6ac5c-93a8-4c3c-a020-0c9aabe9c4b2&search_type=unknown&pagination_search=true&cursor=eyJzZWN0aW9uX29mZnNldCI6MywiaXRlbXNfb2Zmc2V0IjoxOCwidmVyc2lvbiI6MX0%3D
Avanzando a la página 3
URL actual: https://www.airbnb.es/s/Madrid--Espa%C3%B1a/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&query=Madrid%2C%20Espa%C3%B1a&place_id=ChIJgTwKgJcpQg0RaSKMYcHeNsQ&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2024-03-01&monthly_length=3&monthly_end_date=2024-06-01&price_filter_input_type=0&price_filter_num_nights=5&chann

In [37]:
#@title boton 13guardar
from datetime import datetime

# Limpiar los saltos de línea y comas en las columnas de texto
df['titulo'] = df['titulo'].str.replace('\n', ' ').str.replace(',', '')
df['subtitulo'] = df['subtitulo'].str.replace('\n', ' ').str.replace(',', '')
# Repetir para cualquier otra columna que pueda tener este problema


# Ruta donde deseas guardar el archivo CSV en Google Drive (nombre único con marca de tiempo)
timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
nombre_archivo_unico = f'anuncios_airbnb_madrid_pag13_{timestamp}.csv'
ruta_guardado_unico = f'/content/drive/MyDrive/Hosting/Hosting/Analisis Scrapping/csv/{nombre_archivo_unico}'

# Guardar el DataFrame en un archivo CSV único basado en la marca de tiempo en Google Drive
df.to_csv(ruta_guardado_unico, sep='\t', encoding='utf-16', index=False)
print(f"Archivo CSV único con marca de tiempo guardado en: {ruta_guardado_unico}")

Archivo CSV único con marca de tiempo guardado en: /content/drive/MyDrive/Hosting/Hosting/Analisis Scrapping/csv/anuncios_airbnb_madrid_pag13_20240214112359.csv


In [38]:
#@title botón 14
# import time # Librería para controlar el tiempo
# from selenium import webdriver # Librería para controlar el navegador
# from selenium.webdriver.chrome.options import Options # Librería para controlar el navegador
# from selenium.webdriver.common.by import By # Librería para controlar el navegador
# from selenium.webdriver.support.ui import WebDriverWait # Librería para controlar el navegador
# from selenium.webdriver.support import expected_conditions as EC #  Librería para controlar el navegador
# from fake_useragent import UserAgent # Librería para controlar el navegador
# import pandas as pd # Librería para controlar el navegador

# # Configuración inicial del WebDriver y opciones
# ua = UserAgent() # Crear un objeto UserAgent
# chrome_options = Options() # Crear un objeto Options
# chrome_options.add_argument('--headless') # Modo headless
# chrome_options.add_argument('--no-sandbox')
# chrome_options.add_argument('--disable-dev-shm-usage')
# chrome_options.add_argument("--disable-extensions")
# chrome_options.add_argument(f'user-agent={ua.random}')
# chrome_options.headless = True #
# driver = webdriver.Chrome(options=chrome_options) # Crear un objeto WebDriver


# URL de la primera página de Airbnb
airbnb_url_madrid = 'https://www.airbnb.es/s/Madrid--España/homes'
driver.get(airbnb_url_madrid)
time.sleep(5)
driver.implicitly_wait(20)

print("URL inicial:", driver.current_url)  # Imprimir la URL inicial

# Navegación a la página deseada (Página 14)
pagina_deseada = 14
pagina_actual = 1

while pagina_actual < pagina_deseada:
    try:
        boton_siguiente = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[contains(@aria-label, 'Siguiente')]"))
        )
        boton_siguiente.click()
        time.sleep(5)
        print(f"Avanzando a la página {pagina_actual + 1}")
        pagina_actual += 1
        print("URL actual:", driver.current_url)  # Imprimir la URL después de cada cambio de página
    except Exception as e:
        print(f"No se pudo hacer clic en el botón Siguiente: {e}")
        break

# Verificación de llegada a la página deseada y extracción de datos
if pagina_actual == pagina_deseada:
    print(f"Se ha llegado a la página {pagina_deseada}.")
    print("URL actual:", driver.current_url)  # Imprimir la URL en la página deseada
    anuncios_pisos_pag14 = []

    try:
        WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'c4mnd7m')))
        elementos_anunciospag14 = driver.find_elements(By.CLASS_NAME, 'c4mnd7m')
            # Esperas para elementos adicionales en el DOM
        precios_noche = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CLASS_NAME, '_1jo4hgw')))
        subtitulos_anuncio = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 't6mzqp7')))
        valoraciones_anuncio = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".r1dxllyb.atm_7l_18pqv07.atm_cp_1ts48j8.dir.dir-ltr")))
        titulos_anuncio = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 't1jojoys')))
        precios_total = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CLASS_NAME, '_tt122m')))

        for index, anunciopag14 in enumerate(elementos_anunciospag14):
            enlace_anuncio = anunciopag14.find_element(By.TAG_NAME, 'a')
            href_anuncio = enlace_anuncio.get_attribute('href')
            target_anuncio = enlace_anuncio.get_attribute('target')


            # Extraer información usando el índice correspondiente
            titulo = titulos_anuncio[index].text if index < len(titulos_anuncio) else 'N/A'
            subtitulo = subtitulos_anuncio[index].text if index < len(subtitulos_anuncio) else 'N/A'
            valoracion = valoraciones_anuncio[index].text if index < len(valoraciones_anuncio) else 'N/A'
            precio_noche = precios_noche[index].text if index < len(precios_noche) else 'N/A'
            precio_total = precios_total[index].text if index < len(precios_total) else 'N/A'

            anuncios_pisos_pag14.append({
                'texto': anunciopag14.text,
                'titulo': titulo,
                'subtitulo': subtitulo,
                'valoracion': valoracion,
                'precio_noche': precio_noche,
                'precio_total': precio_total,
                'href': href_anuncio,
                'target': target_anuncio
            })

        # Intentar realizar scraping solo si se obtuvieron enlaces
        if anuncios_pisos_pag14:
            try:
                for anuncio in anuncios_pisos_pag14:
                    driver.get(anuncio['href'])
                    WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "lgx66tx.atm_gi_idpfg4.atm_l8_idpfg4.dir.dir-ltr")))
                    caracteristicas = driver.find_elements(By.CLASS_NAME, "lgx66tx.atm_gi_idpfg4.atm_l8_idpfg4.dir.dir-ltr")
                    caracteristicas_texto = [c.text for c in caracteristicas]
                    print(f"Características para {anuncio['href']}: {caracteristicas_texto}")
                    anuncio['caracteristicas'] = caracteristicas_texto
            except Exception as e:
                print(f"No se pudo realizar scraping de características para {anuncio['href']}: {e}")

            df = pd.DataFrame(anuncios_pisos_pag14)
            df.to_csv('resultados_airbnb_pag14.csv', index=False)
            print("Resultados finales:", df)

    except Exception as e:
        print(f"Error al extraer anuncios: {e}")
else:
    print("No se obtuvieron enlaces, no se realiza scraping.")


URL inicial: https://www.airbnb.es/s/Madrid--Espa%C3%B1a/homes
Avanzando a la página 2
URL actual: https://www.airbnb.es/s/Madrid--Espa%C3%B1a/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&query=Madrid%2C%20Espa%C3%B1a&place_id=ChIJgTwKgJcpQg0RaSKMYcHeNsQ&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2024-03-01&monthly_length=3&monthly_end_date=2024-06-01&price_filter_input_type=0&price_filter_num_nights=5&channel=EXPLORE&federated_search_session_id=c4dcdf0c-b89e-4fab-8897-c6631b238f0f&search_type=unknown&pagination_search=true&cursor=eyJzZWN0aW9uX29mZnNldCI6MywiaXRlbXNfb2Zmc2V0IjoxOCwidmVyc2lvbiI6MX0%3D
Avanzando a la página 3
URL actual: https://www.airbnb.es/s/Madrid--Espa%C3%B1a/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&query=Madrid%2C%20Espa%C3%B1a&place_id=ChIJgTwKgJcpQg0RaSKMYcHeNsQ&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2024-03-01&monthly_length=3&monthly_end_date=2024-06-01&price_filter_input_type=0&price_filter_num_nights=5&chann

In [39]:
#@title boton 14guardar
from datetime import datetime

# Limpiar los saltos de línea y comas en las columnas de texto
df['titulo'] = df['titulo'].str.replace('\n', ' ').str.replace(',', '')
df['subtitulo'] = df['subtitulo'].str.replace('\n', ' ').str.replace(',', '')
# Repetir para cualquier otra columna que pueda tener este problema


# Ruta donde deseas guardar el archivo CSV en Google Drive (nombre único con marca de tiempo)
timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
nombre_archivo_unico = f'anuncios_airbnb_madrid_pag14_{timestamp}.csv'
ruta_guardado_unico = f'/content/drive/MyDrive/Hosting/Hosting/Analisis Scrapping/csv/{nombre_archivo_unico}'

# Guardar el DataFrame en un archivo CSV único basado en la marca de tiempo en Google Drive
df.to_csv(ruta_guardado_unico, sep='\t', encoding='utf-16', index=False)
print(f"Archivo CSV único con marca de tiempo guardado en: {ruta_guardado_unico}")

Archivo CSV único con marca de tiempo guardado en: /content/drive/MyDrive/Hosting/Hosting/Analisis Scrapping/csv/anuncios_airbnb_madrid_pag14_20240214112740.csv


In [40]:
#@title botón 15

# import time # Librería para controlar el tiempo
# from selenium import webdriver # Librería para controlar el navegador
# from selenium.webdriver.chrome.options import Options # Librería para controlar el navegador
# from selenium.webdriver.common.by import By # Librería para controlar el navegador
# from selenium.webdriver.support.ui import WebDriverWait # Librería para controlar el navegador
# from selenium.webdriver.support import expected_conditions as EC #  Librería para controlar el navegador
# from fake_useragent import UserAgent # Librería para controlar el navegador
# import pandas as pd # Librería para controlar el navegador

# # Configuración inicial del WebDriver y opciones
# ua = UserAgent() # Crear un objeto UserAgent
# chrome_options = Options() # Crear un objeto Options
# chrome_options.add_argument('--headless') # Modo headless
# chrome_options.add_argument('--no-sandbox')
# chrome_options.add_argument('--disable-dev-shm-usage')
# chrome_options.add_argument("--disable-extensions")
# chrome_options.add_argument(f'user-agent={ua.random}')
# chrome_options.headless = True #
# driver = webdriver.Chrome(options=chrome_options) # Crear un objeto WebDriver


# URL de la primera página de Airbnb
airbnb_url_madrid = 'https://www.airbnb.es/s/Madrid--España/homes'
driver.get(airbnb_url_madrid)
time.sleep(5)
driver.implicitly_wait(20)

print("URL inicial:", driver.current_url)  # Imprimir la URL inicial

# Navegación a la página deseada (Página 15)
pagina_deseada = 15
pagina_actual = 1

while pagina_actual < pagina_deseada:
    try:
        boton_siguiente = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[contains(@aria-label, 'Siguiente')]"))
        )
        boton_siguiente.click()
        time.sleep(5)
        print(f"Avanzando a la página {pagina_actual + 1}")
        pagina_actual += 1
        print("URL actual:", driver.current_url)  # Imprimir la URL después de cada cambio de página
    except Exception as e:
        print(f"No se pudo hacer clic en el botón Siguiente: {e}")
        break

# Verificación de llegada a la página deseada y extracción de datos
if pagina_actual == pagina_deseada:
    print(f"Se ha llegado a la página {pagina_deseada}.")
    print("URL actual:", driver.current_url)  # Imprimir la URL en la página deseada
    anuncios_pisos_pag15 = []


    try:
        WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'c4mnd7m')))
        elementos_anunciospag15 = driver.find_elements(By.CLASS_NAME, 'c4mnd7m')
            # Esperas para elementos adicionales en el DOM
        precios_noche = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CLASS_NAME, '_1jo4hgw')))
        subtitulos_anuncio = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 't6mzqp7')))
        valoraciones_anuncio = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".r1dxllyb.atm_7l_18pqv07.atm_cp_1ts48j8.dir.dir-ltr")))
        titulos_anuncio = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 't1jojoys')))
        precios_total = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CLASS_NAME, '_tt122m')))

        for index, anunciopag15 in enumerate(elementos_anunciospag15):
            enlace_anuncio = anunciopag15.find_element(By.TAG_NAME, 'a')
            href_anuncio = enlace_anuncio.get_attribute('href')
            target_anuncio = enlace_anuncio.get_attribute('target')


            # Extraer información usando el índice correspondiente
            titulo = titulos_anuncio[index].text if index < len(titulos_anuncio) else 'N/A'
            subtitulo = subtitulos_anuncio[index].text if index < len(subtitulos_anuncio) else 'N/A'
            valoracion = valoraciones_anuncio[index].text if index < len(valoraciones_anuncio) else 'N/A'
            precio_noche = precios_noche[index].text if index < len(precios_noche) else 'N/A'
            precio_total = precios_total[index].text if index < len(precios_total) else 'N/A'

            anuncios_pisos_pag15.append({
                'texto': anunciopag15.text,
                'titulo': titulo,
                'subtitulo': subtitulo,
                'valoracion': valoracion,
                'precio_noche': precio_noche,
                'precio_total': precio_total,
                'href': href_anuncio,
                'target': target_anuncio
            })

        # Intentar realizar scraping solo si se obtuvieron enlaces
        if anuncios_pisos_pag15:
            try:
                for anuncio in anuncios_pisos_pag15:
                    driver.get(anuncio['href'])
                    WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "lgx66tx.atm_gi_idpfg4.atm_l8_idpfg4.dir.dir-ltr")))
                    caracteristicas = driver.find_elements(By.CLASS_NAME, "lgx66tx.atm_gi_idpfg4.atm_l8_idpfg4.dir.dir-ltr")
                    caracteristicas_texto = [c.text for c in caracteristicas]
                    print(f"Características para {anuncio['href']}: {caracteristicas_texto}")
                    anuncio['caracteristicas'] = caracteristicas_texto
            except Exception as e:
                print(f"No se pudo realizar scraping de características para {anuncio['href']}: {e}")

            df = pd.DataFrame(anuncios_pisos_pag15)
            df.to_csv('resultados_airbnb_pag15.csv', index=False)
            print("Resultados finales:", df)

    except Exception as e:
        print(f"Error al extraer anuncios: {e}")
else:
    print("No se obtuvieron enlaces, no se realiza scraping.")


URL inicial: https://www.airbnb.es/s/Madrid--Espa%C3%B1a/homes
Avanzando a la página 2
URL actual: https://www.airbnb.es/s/Madrid--Espa%C3%B1a/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&query=Madrid%2C%20Espa%C3%B1a&place_id=ChIJgTwKgJcpQg0RaSKMYcHeNsQ&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2024-03-01&monthly_length=3&monthly_end_date=2024-06-01&price_filter_input_type=0&price_filter_num_nights=5&channel=EXPLORE&federated_search_session_id=01ba8402-2d42-4927-afa1-e5c8233cf16f&search_type=unknown&pagination_search=true&cursor=eyJzZWN0aW9uX29mZnNldCI6MywiaXRlbXNfb2Zmc2V0IjoxOCwidmVyc2lvbiI6MX0%3D
Avanzando a la página 3
URL actual: https://www.airbnb.es/s/Madrid--Espa%C3%B1a/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&query=Madrid%2C%20Espa%C3%B1a&place_id=ChIJgTwKgJcpQg0RaSKMYcHeNsQ&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2024-03-01&monthly_length=3&monthly_end_date=2024-06-01&price_filter_input_type=0&price_filter_num_nights=5&chann

In [41]:
#@title boton 15 guardar
from datetime import datetime

# Limpiar los saltos de línea y comas en las columnas de texto
df['titulo'] = df['titulo'].str.replace('\n', ' ').str.replace(',', '')
df['subtitulo'] = df['subtitulo'].str.replace('\n', ' ').str.replace(',', '')
# Repetir para cualquier otra columna que pueda tener este problema


# Ruta donde deseas guardar el archivo CSV en Google Drive (nombre único con marca de tiempo)
timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
nombre_archivo_unico = f'anuncios_airbnb_madrid_pag15_{timestamp}.csv'
ruta_guardado_unico = f'/content/drive/MyDrive/Hosting/Hosting/Analisis Scrapping/csv/{nombre_archivo_unico}'

# Guardar el DataFrame en un archivo CSV único basado en la marca de tiempo en Google Drive
df.to_csv(ruta_guardado_unico, sep='\t', encoding='utf-16', index=False)
print(f"Archivo CSV único con marca de tiempo guardado en: {ruta_guardado_unico}")

Archivo CSV único con marca de tiempo guardado en: /content/drive/MyDrive/Hosting/Hosting/Analisis Scrapping/csv/anuncios_airbnb_madrid_pag15_20240214113138.csv


## guardar en un csv todos los anuncios

In [42]:
# Lista principal para almacenar la información de todos los anuncios
todos_los_anuncios = []

todos_los_anuncios.extend(anuncios_pisos_pag1)
todos_los_anuncios.extend(anuncios_pisos_pag2)
todos_los_anuncios.extend(anuncios_pisos_pag3)
todos_los_anuncios.extend(anuncios_pisos_pag4)
todos_los_anuncios.extend(anuncios_pisos_pag5)
todos_los_anuncios.extend(anuncios_pisos_pag6)
todos_los_anuncios.extend(anuncios_pisos_pag7)
todos_los_anuncios.extend(anuncios_pisos_pag8)
todos_los_anuncios.extend(anuncios_pisos_pag9)
todos_los_anuncios.extend(anuncios_pisos_pag10)
todos_los_anuncios.extend(anuncios_pisos_pag11)
todos_los_anuncios.extend(anuncios_pisos_pag12)
todos_los_anuncios.extend(anuncios_pisos_pag13)
todos_los_anuncios.extend(anuncios_pisos_pag14)
todos_los_anuncios.extend(anuncios_pisos_pag15)
# Crear un DataFrame a partir de la lista de todos los anuncios
df_anuncios = pd.DataFrame(todos_los_anuncios)

# Limpiar los saltos de línea y comas en las columnas de texto
df_anuncios['titulo'] = df_anuncios['titulo'].str.replace('\n', ' ').str.replace(',', '')
df_anuncios['subtitulo'] = df_anuncios['subtitulo'].str.replace('\n', ' ').str.replace(',', '')
# Repetir para cualquier otra columna que pueda tener este problema


# Ruta donde deseas guardar el archivo CSV (nombre único con marca de tiempo)
timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
nombre_archivo_unico = f'anuncios_airbnb_madrid_{timestamp}.csv'
ruta_guardado = f'/content/drive/MyDrive/Hosting/Hosting/Analisis Scrapping/csv/{nombre_archivo_unico}'  # Ajusta esta ruta según sea necesario

# Guardar el DataFrame en un archivo CSV único basado en la marca de tiempo
df_anuncios.to_csv(ruta_guardado, sep='\t', encoding='utf-16', index=False)
print(f"Archivo CSV único con marca de tiempo guardado en: {ruta_guardado}")

# Mostrar la ruta del archivo guardado para acceso del usuario
ruta_guardado


Archivo CSV único con marca de tiempo guardado en: /content/drive/MyDrive/Hosting/Hosting/Analisis Scrapping/csv/anuncios_airbnb_madrid_20240214113138.csv


'/content/drive/MyDrive/Hosting/Hosting/Analisis Scrapping/csv/anuncios_airbnb_madrid_20240214113138.csv'

# 7.-Extracción elementos BOTÓN MÁS SERVICIOS del primer enlace

In [43]:
# # Obtener la URL del primer enlace
# primer_enlace = enlaces[0]['url']

# # Navegar a la URL con Selenium
# driver.get(primer_enlace)

# # Esperar un tiempo fijo (ajustar según sea necesario)
# time.sleep(5)

# # Obtener el HTML de la página después de hacer clic en el enlace
# html_pagina_individual = driver.page_source

# # Parsear el HTML de la página individual
# soup_pagina_individual = BeautifulSoup(html_pagina_individual, 'lxml')

# # Encuentra lOS BOTONES de pisos en la página individual
# listings_pisos_individual_boton = soup_pagina_individual.find_all('button', {'class': ["l1ovpqvx atm_1y33qqm_1ggndnn_10saat9 atm_17zvjtw_zk357r_10saat9 atm_w3cb4q_il40rs_10saat9 b65jmrv atm_9j_tlke0l atm_9s_1o8liyq atm_gi_idpfg4 atm_mk_h2mmj6 atm_r3_1h6ojuz atm_rd_glywfm atm_3f_97hwo atm_vy_1wugsn5 atm_tl_1gw4zv3 atm_9j_13gfvf7_1o5j5ji v7aged4 atm_bx_1ltc5j7 atm_c8_8ycq01 atm_g3_adnk3f atm_cs_qo5vgd atm_5j_9l7fl4 atm_6h_t94yts atm_66_nqa18y atm_uc_1c5iv21 atm_kd_glywfm atm_4b_18pqv07 atm_26_1hbpp16 atm_7l_18pqv07 atm_l8_1vkzbvs atm_uc_glywfm__p88qr9 atm_3f_glywfm_jo46a5 atm_l8_idpfg4_jo46a5 atm_gi_idpfg4_jo46a5 atm_3f_glywfm_1icshfk atm_kd_glywfm_19774hq atm_uc_x37zl0_1w3cfyq atm_uc_glywfm_1w3cfyq_p88qr9 atm_uc_x37zl0_18zk5v0 atm_uc_glywfm_18zk5v0_p88qr9 atm_tr_18md41p_csw3t1 atm_k4_kb7nvz_1o5j5ji atm_4b_18pqv07_1w3cfyq atm_7l_18pqv07_1w3cfyq atm_70_216vci_1w3cfyq atm_4b_18pqv07_18zk5v0 atm_7l_18pqv07_18zk5v0 atm_70_216vci_18zk5v0 atm_4b_1otlplk_1nos8r atm_26_1nh1gcj_1nos8r atm_7l_18pqv07_1nos8r atm_4b_1otlplk_csw3t1 atm_26_1nh1gcj_csw3t1 atm_7l_18pqv07_csw3t1 atm_4b_161hw1_1o5j5ji atm_26_1hbpp16_1o5j5ji atm_7l_161hw1_1o5j5ji dir dir-ltr"]})

# # listings_pisos_individual_boton = soup_pagina_individual.find_all('div', {'class': ["b9672i7 atm_h3_xvenqj atm_h3_f13iio__oggzyc dir dir-ltr"]})

# # Por ejemplo, imprimir el texto de cada elemento en listings_pisos_individual
# for listing in listings_pisos_individual_boton:
#     print(listing.get_text())


In [44]:
# Encuentra el botón específico con la clase mencionada
boton_especifico = soup.find('button', class_="l1ovpqvx atm_1y33qqm_1ggndnn_10saat9 atm_17zvjtw_zk357r_10saat9 atm_w3cb4q_il40rs_10saat9 b65jmrv atm_9j_tlke0l atm_9s_1o8liyq atm_gi_idpfg4 atm_mk_h2mmj6 atm_r3_1h6ojuz atm_rd_glywfm atm_3f_97hwo atm_vy_1wugsn5 atm_tl_1gw4zv3 atm_9j_13gfvf7_1o5j5ji v7aged4 atm_bx_1ltc5j7 atm_c8_8ycq01 atm_g3_adnk3f atm_cs_qo5vgd atm_5j_9l7fl4 atm_6h_t94yts atm_66_nqa18y atm_uc_1c5iv21 atm_kd_glywfm atm_4b_18pqv07 atm_26_1hbpp16 atm_7l_18pqv07 atm_l8_1vkzbvs atm_uc_glywfm__p88qr9 atm_3f_glywfm_jo46a5 atm_l8_idpfg4_jo46a5 atm_gi_idpfg4_jo46a5 atm_3f_glywfm_1icshfk atm_kd_glywfm_19774hq atm_uc_x37zl0_1w3cfyq atm_uc_glywfm_1w3cfyq_p88qr9 atm_uc_x37zl0_18zk5v0 atm_uc_glywfm_18zk5v0_p88qr9 atm_tr_18md41p_csw3t1 atm_k4_kb7nvz_1o5j5ji atm_4b_18pqv07_1w3cfyq atm_7l_18pqv07_1w3cfyq atm_70_216vci_1w3cfyq atm_4b_18pqv07_18zk5v0 atm_7l_18pqv07_18zk5v0 atm_70_216vci_18zk5v0 atm_4b_1otlplk_1nos8r atm_26_1nh1gcj_1nos8r atm_7l_18pqv07_1nos8r atm_4b_1otlplk_csw3t1 atm_26_1nh1gcj_csw3t1 atm_7l_18pqv07_csw3t1 atm_4b_161hw1_1o5j5ji atm_26_1hbpp16_1o5j5ji atm_7l_161hw1_1o5j5ji dir dir-ltr")

# Extrae la información que necesitas del botón
if boton_especifico:
    texto_del_boton = boton_especifico.get_text()
    # Realiza las acciones que necesites con la información extraída
    print(texto_del_boton)
else:
    print("No se encontró el botón específico.")

No se encontró el botón específico.


##### Opcion 1

In [45]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
#Espera a que el elemento sea visible:
wait = WebDriverWait(driver, 10)
elemento = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="site-content"]/div/div[1]/div[3]/div/div[1]/div/div[8]/div/div[2]/section/div[3]/button')))
elemento.click()



TimeoutException: Message: 
Stacktrace:
#0 0x56d8a33edd93 <unknown>
#1 0x56d8a30d1337 <unknown>
#2 0x56d8a311cd60 <unknown>
#3 0x56d8a311ce51 <unknown>
#4 0x56d8a3160bc4 <unknown>
#5 0x56d8a313f46d <unknown>
#6 0x56d8a315e12d <unknown>
#7 0x56d8a313f1e3 <unknown>
#8 0x56d8a310f135 <unknown>
#9 0x56d8a311013e <unknown>
#10 0x56d8a33b1e4b <unknown>
#11 0x56d8a33b5dfa <unknown>
#12 0x56d8a339e6d5 <unknown>
#13 0x56d8a33b6a6f <unknown>
#14 0x56d8a338269f <unknown>
#15 0x56d8a33db098 <unknown>
#16 0x56d8a33db262 <unknown>
#17 0x56d8a33ecf34 <unknown>
#18 0x7e0d0cc3eac3 <unknown>


In [ ]:
#Asegúrate de que el elemento esté habilitado:
elemento = driver.find_element(By.XPATH, '//*[@id="site-content"]/div/div[1]/div[3]/div/div[1]/div/div[8]/div/div[2]/section/div[3]/button')
if elemento.is_enabled():
    elemento.click()


In [ ]:
# # Si el problema persiste, intenta desplazar el elemento a la vista antes de hacer clic:
# elemento = driver.find_element(By.XPATH, '//*[@id="site-content"]/div/div[1]/div[3]/div/div[1]/div/div[8]/div/div[2]/section/div[3]/button')
# driver.execute_script("arguments[0].scrollIntoView(true);", elemento)
# elemento.click()


In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
#Espera a que el elemento sea interactable utilizando element_to_be_clickable:
wait = WebDriverWait(driver, 10)
elemento = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="site-content"]/div/div[1]/div[3]/div/div[1]/div/div[8]/div/div[2]/section/div[3]/button')))
elemento.click()


##### Opcion 2

In [ ]:
# Intenta hacer clic directamente en el elemento utilizando JavaScript:
elemento = driver.find_element(By.XPATH, '//*[@id="site-content"]/div/div[1]/div[3]/div/div[1]/div/div[8]/div/div[2]/section/div[3]/button')
driver.execute_script("arguments[0].click();", elemento)


In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Espera a que el elemento sea presente en el DOM
elemento = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="site-content"]/div/div[1]/div[3]/div/div[1]/div/div[8]/div/div[2]/section/div[3]/button')))

# Desplaza el elemento a la vista
driver.execute_script("arguments[0].scrollIntoView(true);", elemento)

# Espera a que el elemento sea clickeable utilizando JavaScript
driver.execute_script("arguments[0].click();", elemento)


ambos códigos realizan clic en el botón identificado por el XPath proporcionado.

In [ ]:
# Espera a que la cantidad correcta de ventanas esté presente
wait = WebDriverWait(driver, 10)
wait.until(EC.number_of_windows_to_be(10))  # Ajusta el número según sea necesario

# Cambia al controlador de la nueva ventana
ventanas = driver.window_handles
driver.switch_to.window(ventanas[1])


##### Opcion 3

In [ ]:
try:
    # Intento de hacer clic en el botón de amenities
    boton_especifico = driver.find_element(By.CLASS_NAME, 'b9672i7')
    actions = ActionChains(driver)
    driver.execute_script("arguments[0].scrollIntoView(true);", boton_especifico)
    actions.move_to_element_with_offset(boton_especifico, 5, 5)
    actions.click().perform()
except Exception as e:
    print(f"No se pudo hacer clic en el botón de amenities: {e}")


Asegúrate de que el botón sea visible:
Añade una espera explícita para asegurarte de que el botón sea visible antes de intentar hacer clic. Puedes utilizar WebDriverWait para esperar hasta que el elemento sea clickeable.

In [ ]:
try:
    boton_especifico = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CLASS_NAME, 'b9672i7'))
    )
    boton_especifico.click()
except Exception as e:
    print(f"No se pudo hacer clic en el botón de amenities: {e}")

Asegúrate de que el elemento esté dentro del área visual:
Usa scrollIntoView para asegurarte de que el botón esté dentro del área visual antes de hacer clic.

In [ ]:
try:
    # Intento de hacer clic en el botón de amenities
    boton_especifico = driver.find_element(By.CLASS_NAME, 'b9672i7')
    driver.execute_script("arguments[0].scrollIntoView(true);", boton_especifico)
    boton_especifico.click()
except Exception as e:
    print(f"No se pudo hacer clic en el botón de amenities: {e}")

Espera antes de hacer clic:
Añade un breve tiempo de espera antes de intentar hacer clic para asegurarte de que todos los elementos de la página estén completamente cargados.

In [ ]:
import time

try:
    time.sleep(2)  # Espera 2 segundos antes de hacer clic
    boton_especifico = driver.find_element(By.CLASS_NAME, 'b9672i7')
    boton_especifico.click()
except Exception as e:
    print(f"No se pudo hacer clic en el botón de amenities: {e}")


Hacer clic con JavaScript:
A veces, hacer clic con JavaScript puede ser más efectivo. Puedes intentar hacer clic en el botón usando JavaScript.

In [ ]:
try:
    boton_especifico = driver.find_element(By.CLASS_NAME, 'b9672i7')
    driver.execute_script("arguments[0].click();", boton_especifico)
except Exception as e:
    print(f"No se pudo hacer clic en el botón de amenities: {e}")


In [ ]:
boton_especifico

Espera antes de hacer clic: Puede ser útil agregar una pequeña espera antes de intentar hacer clic en el botón. A veces, la página puede tardar un momento en cargar completamente los elementos.

In [ ]:
time.sleep(2)  # Espera 2 segundos antes de hacer clic


Ahora que ya he hehco click y he añadido tiempo de epsera a que se abra la nueva ventana, debo cambiar el controlador a la nueva ventana.

In [ ]:
# Obtener las manijas de todas las ventanas abiertas
ventanas = driver.window_handles
# Toma una captura de pantalla y guárdala como "captura_pantalla.png"
driver.save_screenshot("captura_pantalla.png")

In [ ]:
ventanas

In [ ]:
# Cambiar al controlador de la nueva ventana
driver.switch_to.window(ventanas[0])  # Puedes ajustar el índice según sea necesario
# Toma una captura de pantalla y guárdala como "captura_pantalla.png"
driver.save_screenshot("captura_pantalla2.png")

In [ ]:
# Extraer los elementos que necesitas en la nueva ventana
elementos_nueva_ventana = driver.find_elements(By.XPATH, '//div[@class="_11jhslp"]')

# Ahora puedes trabajar con elementos_nueva_ventana, por ejemplo, imprimir el texto
for elemento in elementos_nueva_ventana:
    print(elemento.text)

In [ ]:
# Inicializamos una lista vacía para almacenar la información
servicios = []

# Iteramos sobre los elementos y almacenamos la información en la lista
for elemento in elementos_nueva_ventana:
    servicios.append(elemento)

# Ahora, la lista contiene la información en el mismo orden en que aparece en elementos_nueva_ventana
print(servicios)